In [12]:
import torch
from transformers import AutoModelForCausalLM,AutoTokenizer
from torch import nn
import time
import gc

In [13]:
model=AutoModelForCausalLM.from_pretrained("meta-llama/Llama-3.2-1B").to('cuda')
question="hi"
tok=AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-1B")
# prompt=tok.apply_chat_template(converstation)
prompt=tok(question,return_tensors='pt').to('cuda')
out=model.generate(**prompt,max_new_tokens=10)


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


In [14]:
out=model.generate(**prompt,max_new_tokens=10)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


In [15]:
out[0]

tensor([128000,   6151,     11,    358,   1097,   3411,    369,   1063,   1520,
           449,    264,   3488], device='cuda:0')

In [ ]:
decoded = tok.decode(out[0], skip_special_tokens=True)
print(decoded)


hi, I am looking for some help with a question


In [17]:
model=AutoModelForCausalLM.from_pretrained("meta-llama/Llama-3.2-1B").to('cpu')
print(model)


LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 2048)
    (layers): ModuleList(
      (0-15): 16 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=512, bias=False)
          (v_proj): Linear(in_features=2048, out_features=512, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=2048, out_features=8192, bias=False)
          (up_proj): Linear(in_features=2048, out_features=8192, bias=False)
          (down_proj): Linear(in_features=8192, out_features=2048, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
      )
    )
    (norm): LlamaRMSNorm((2048,), eps=1e-05)
    (rotary_emb):

In [18]:
model.model.rotary_emb

LlamaRotaryEmbedding()

In [19]:
for i in model.model.layers:
    print(i)
    break

LlamaDecoderLayer(
  (self_attn): LlamaAttention(
    (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
    (k_proj): Linear(in_features=2048, out_features=512, bias=False)
    (v_proj): Linear(in_features=2048, out_features=512, bias=False)
    (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
  )
  (mlp): LlamaMLP(
    (gate_proj): Linear(in_features=2048, out_features=8192, bias=False)
    (up_proj): Linear(in_features=2048, out_features=8192, bias=False)
    (down_proj): Linear(in_features=8192, out_features=2048, bias=False)
    (act_fn): SiLU()
  )
  (input_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
  (post_attention_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
)


In [20]:
dtype = torch.bfloat16
bytes_per_param = torch.tensor([], dtype=dtype).element_size()
bytes_per_param

2

In [21]:
l=model.model.layers[0]
num_params = sum(p.numel() for p in l.parameters())
num_params
param_size_bytes = num_params * 4
param_size_MB = param_size_bytes / (1024 ** 3)
param_size_MB 

0.2265777587890625

In [22]:
(torch.cuda.get_device_properties('cuda').total_memory)/(1024**3)

7.6064453125

In [23]:
int((torch.cuda.get_device_properties('cuda').total_memory)/(1024**3)/param_size_MB )

33

In [24]:
layers=model.model.layers
len(layers)

16

In [25]:
layers=layers[:-1]
len(layers)

15

In [26]:
def get_layers(model,drop=2):
    device_memory=(torch.cuda.get_device_properties('cuda').total_memory)/(1024**3)
    l=model.model.layers[0]
    num_params = sum(p.numel() for p in l.parameters())
    param_size_bytes = num_params * 4
    param_size_GB = param_size_bytes / (1024 ** 3)
    return int(device_memory/param_size_GB)-drop

In [27]:
get_layers(model)

31

In [28]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from torch import nn
import time
import gc

# Enable CUDA and Tensor Core optimizations
torch.backends.cudnn.benchmark = True
torch.backends.cuda.matmul.allow_tf32 = True

# Load the model and tokenizer
# Corrected the missing closing quote in the tokenizer
model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-3.2-1B",
                                             torch_dtype=torch.bfloat16,
                                             use_cache=True,
                                             # quantization_config=bnb_config
                                             )
model.to('cpu')

tok = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-1B")


def get_layers(model, drop=2):
    """
    Calculates the maximum number of layers that can fit on the GPU.
    This function has been improved to account for static model parts
    and provide a more robust chunk size calculation.

    Args:
        model: The model instance.
        drop (int): A memory buffer in GB to leave on the GPU.

    Returns:
        int: The number of layers per chunk.
    """
    device_memory = (torch.cuda.get_device_properties('cuda').total_memory) / (1024**3)
    l = model.model.layers[0]
    num_params = sum(p.numel() for p in l.parameters())
    param_size_bytes = num_params * 2 # Using bfloat16 for correct size calculation
    param_size_GB = param_size_bytes / (1024 ** 3)
    
    # Adjusting for static model parts which are loaded once
    static_mem_gb = 0
    if hasattr(model.model, 'embed_tokens'):
        static_mem_gb += sum(p.numel() for p in model.model.embed_tokens.parameters()) * 2 / (1024**3)
    if hasattr(model.model, 'norm'):
        static_mem_gb += sum(p.numel() for p in model.model.norm.parameters()) * 2 / (1024**3)
    if hasattr(model, 'lm_head'):
        static_mem_gb += sum(p.numel() for p in model.lm_head.parameters()) * 2 / (1024**3)

    available_gpu_memory_for_layers_gb = device_memory - drop - static_mem_gb
    if available_gpu_memory_for_layers_gb <= 0:
        print(f"Warning: Not enough GPU memory. Available: {device_memory:.2f} GB. Falling back to 1 layer per chunk.")
        return 1
    
    if param_size_GB == 0:
        return 1

    return max(1, int(available_gpu_memory_for_layers_gb / param_size_GB))


class Module(nn.Module):
    """A wrapper for a chunk of layers."""
    def __init__(self, layers, i, j):
        super().__init__()
        self.model = nn.ModuleList(layers[i:j])
    
    @torch.compile
    def forward(self, x, cos, sin, attention_mask):
        for layer in self.model:
            # Pass the attention mask to the layer
            x = layer(x, position_embeddings=(cos, sin), attention_mask=attention_mask)[0]
        return x


def compute(model_instance, x, layers_per_chunk, cos, sin, k, max_new_tokens, attention_mask):
    """
    Computes a forward pass using a three-worker pipeline with three explicit streams.
    Worker 1: Computation
    Worker 2: Loading new layers
    Worker 3: Offloading old layers
    """
    total_layers = len(model_instance.model.layers)

    stream_compute = torch.cuda.Stream()
    stream_load = torch.cuda.Stream()
    stream_offload = torch.cuda.Stream()

    # Case 1: All layers fit on the GPU, no offloading needed.
    if layers_per_chunk >= total_layers:
        m = Module(model_instance.model.layers, 0, total_layers).to('cuda')
        x = m(x.to('cuda'), cos.to('cuda'), sin.to('cuda'), attention_mask.to('cuda'))
        if k == max_new_tokens - 1:
            m.to('cpu')
            torch.cuda.empty_cache()
            del m
        return x

    # Case 2: Pipelined offloading with three streams
    i = 0
    rem = total_layers
    
    # Priming the pipeline: Load the first chunk on the load stream
    m_current = None
    with torch.cuda.stream(stream_load):
        m_current = Module(model_instance.model.layers, i, i + layers_per_chunk).to('cuda', non_blocking=True)
    i += layers_per_chunk
    rem -= layers_per_chunk

    m_next = None
    
    while True:
        # Load the next chunk if there are layers remaining
        if rem > 0:
            l_next = min(layers_per_chunk, rem)
            with torch.cuda.stream(stream_load):
                m_next = Module(model_instance.model.layers, i, i + l_next).to('cuda', non_blocking=True)
        else:
            m_next = None

        # Wait for the current chunk to finish loading
        stream_compute.wait_stream(stream_load)

        # Compute on the current chunk
        with torch.cuda.stream(stream_compute):
            with torch.no_grad():
                x = m_current(x, cos, sin, attention_mask)

        # Check if this is the last chunk
        if rem <= 0:
            break
        
        # Offload the current chunk asynchronously
        with torch.cuda.stream(stream_offload):
            m_current.to('cpu', non_blocking=True)

        # Advance the pipeline and update counters
        m_current = m_next
        i += l_next
        rem -= l_next

    # Final cleanup after the loop
    stream_compute.synchronize()
    # The last m_current must be moved back to cpu explicitly
    m_current.to('cpu')
    del m_current
    if m_next:
        del m_next
    torch.cuda.empty_cache()
    return x


def model_inference(question, tokens=250, temperature=0.7, top_p=0.95):
    start_total_time = time.time()
    
    model.to('cpu')
    
    prompt = tok(question, return_tensors='pt').to('cpu')
    input_ids = prompt['input_ids'].to('cuda')
    # Correctly move attention mask to CUDA and convert its dtype
    attention_mask = prompt['attention_mask'].to('cuda')
    del prompt

    # Move static parts of the model to GPU once
    model.model.embed_tokens.to('cuda')
    model.model.norm.to('cuda')
    model.lm_head.to('cuda')
    rotary_emb = model.model.rotary_emb
    
    generated_text_tokens = [] 

    for k in range(tokens):
        x = model.model.embed_tokens(input_ids)
        
        seq_len = x.shape[1]
        batch_size = x.shape[0]
        position_ids = torch.arange(seq_len, dtype=torch.long, device=x.device).unsqueeze(0).expand(batch_size, -1)
        cos, sin = rotary_emb(x=x,position_ids=position_ids)

        # Create a 4D attention mask with the correct dtype
        causal_mask = torch.triu(torch.ones((1, seq_len, seq_len), device=x.device, dtype=torch.bool), diagonal=1)
        causal_mask = ~causal_mask
        causal_mask = causal_mask.view(1, 1, seq_len, seq_len)
        causal_mask = causal_mask * attention_mask.view(1, 1, 1, seq_len)
        causal_mask = (causal_mask == 0).float() * torch.finfo(x.dtype).min
        
        with torch.no_grad():
            l1 = get_layers(model, 7)
            # Use the calculated l1 value, not a hardcoded number
            x = compute(model, x, l1, cos, sin, k, tokens, causal_mask)
            
            x = model.model.norm(x)
            x = model.lm_head(x)
        
        # Apply temperature and Top-P sampling
        logits = x[:, -1, :] / temperature
        
        # Sort logits in descending order
        sorted_logits, sorted_indices = torch.sort(logits, descending=True)
        
        # Calculate cumulative probabilities
        cumulative_probs = torch.cumsum(torch.softmax(sorted_logits, dim=-1), dim=-1)
        
        # Find the tokens that are in the nucleus
        sorted_indices_to_remove = cumulative_probs > top_p
        
        # Shift the indices to the right to keep at least one token
        sorted_indices_to_remove[..., 1:] = sorted_indices_to_remove[..., :-1].clone()
        sorted_indices_to_remove[..., 0] = 0
        
        # Mask out the logits that are not in the nucleus
        logits[sorted_indices[sorted_indices_to_remove]] = -float('inf')
        
        # Sample a token from the nucleus
        next_token = torch.multinomial(torch.softmax(logits, dim=-1), num_samples=1)
        
        print(tok.decode(next_token[0]), end="")
        generated_text_tokens.append(tok.decode(next_token[0]))
        
        input_ids = torch.cat([input_ids, next_token.to('cuda')], dim=-1)
        attention_mask = torch.cat([attention_mask, torch.ones(attention_mask.shape[0], 1).to('cuda')], dim=-1)
    
    # Final cleanup at the very end
    model.model.embed_tokens.to('cpu')
    model.model.norm.to('cpu')
    model.lm_head.to('cpu')
    torch.cuda.empty_cache() 
    
    end_total_time = time.time()
    print(f"\nTime taken for {tokens} tokens: {end_total_time - start_total_time:.2f} seconds")
    return "".join(generated_text_tokens)


# Example usage
print("Starting inference...")
start = time.time()
model_inference("HI", 10)
print()
print("Total time", time.time() - start)


Starting inference...


Traceback (most recent call last):
  File "/home/pa/miniconda3/envs/gen_ai/lib/python3.10/site-packages/torch/_inductor/compile_worker/__main__.py", line 10, in <module>
    from torch._inductor.async_compile import pre_fork_setup
  File "/home/pa/miniconda3/envs/gen_ai/lib/python3.10/site-packages/torch/_inductor/async_compile.py", line 17, in <module>
    from torch._dynamo.device_interface import get_registered_device_interfaces
  File "/home/pa/miniconda3/envs/gen_ai/lib/python3.10/site-packages/torch/_dynamo/__init__.py", line 13, in <module>
    from . import config, convert_frame, eval_frame, resume_execution
  File "/home/pa/miniconda3/envs/gen_ai/lib/python3.10/site-packages/torch/_dynamo/convert_frame.py", line 52, in <module>
    from torch._dynamo.symbolic_convert import TensorifyState
  File "/home/pa/miniconda3/envs/gen_ai/lib/python3.10/site-packages/torch/_dynamo/symbolic_convert.py", line 52, in <module>
    from torch._dynamo.exc import TensorifyScalarRestartAnalysis


KeyboardInterrupt: 

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from torch import nn
import time
import gc

# Enable CUDA and Tensor Core optimizations
torch.backends.cudnn.benchmark = True
torch.backends.cuda.matmul.allow_tf32 = True

# Load the model and tokenizer
# Corrected the missing closing quote in the tokenizer
model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-3.2-1B",
                                             torch_dtype=torch.bfloat16,
                                             use_cache=True,
                                             # quantization_config=bnb_config
                                             )
model.to('cpu')

tok = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-1B")

/home/pa/miniconda3/envs/gen_ai/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
import torch
from transformers import AutoModelForCausalLM,AutoTokenizer,BitsAndBytesConfig
from torch import nn
import time
import gc
# Add at the beginning of your script:
torch.backends.cudnn.benchmark = True
torch.backends.cuda.matmul.allow_tf32 = True
bnb_config=BitsAndBytesConfig(
            load_in_8bit=True,
            # bnb_4bit_compute_dtype=torch.bfloat16,
            # bnb_4bit_use_double_quant=True,
            # bnb_8bit_quant_type='nf4',
        )
model=AutoModelForCausalLM.from_pretrained("meta-llama/Llama-3.2-1B",
                                           torch_dtype=torch.bfloat16,
                                           use_cache=True,
                                        #    quantization_config=bnb_config
                                           )
# print(model)
tok=AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-1B")

/home/pa/miniconda3/envs/gen_ai/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [17]:



def get_layers(model,drop=2):
    device_memory=(torch.cuda.get_device_properties('cuda').total_memory)/(1024**3)
    l=model.model.layers[0]
    num_params = sum(p.numel() for p in l.parameters())
    param_size_bytes = num_params * 2
    param_size_GB = param_size_bytes / (1024 ** 3)
    return int(device_memory/param_size_GB)-drop

class Module(nn.Module):
    """A wrapper for a chunk of layers."""
    def __init__(self, layers, i, j):
        super().__init__()
        self.model = nn.ModuleList(layers[i:j])
    
    @torch.compile
    def forward(self, x, cos, sin, attention_mask):
        for layer in self.model:
            # Pass the attention mask to the layer
            x = layer(x, position_embeddings=(cos, sin), attention_mask=attention_mask)[0]
        return x


def compute(model_instance, x, layers_per_chunk, cos, sin, k, max_new_tokens, attention_mask):
    """
    Computes a forward pass using a three-worker pipeline with three explicit streams.
    Worker 1: Computation
    Worker 2: Loading new layers
    Worker 3: Offloading old layers
    """
    total_layers = len(model_instance.model.layers)

    stream_compute = torch.cuda.Stream()
    stream_load = torch.cuda.Stream()
    stream_offload = torch.cuda.Stream()

    # Case 1: All layers fit on the GPU, no offloading needed.
    if layers_per_chunk >= total_layers:
        m = Module(model_instance.model.layers, 0, total_layers).to('cuda')
        x = m(x.to('cuda'), cos.to('cuda'), sin.to('cuda'), attention_mask.to('cuda'))
        if k == max_new_tokens - 1:
            m.to('cpu')
            torch.cuda.empty_cache()
            del m
        return x

    # Case 2: Pipelined offloading with a robust, explicit three-stage loop.
    i = 0
    
    # Priming the pipeline: Load the first chunk
    m_current = None
    with torch.cuda.stream(stream_load):
        m_current = Module(model_instance.model.layers, i, i + layers_per_chunk).to('cuda', non_blocking=True)
    i += layers_per_chunk
    
    # Initialize offload module to None for the first iteration
    m_to_offload = None
    
    while i <= total_layers:
        # Load the next chunk if there are layers remaining
        m_next = None
        if i < total_layers:
            l_next = min(layers_per_chunk, total_layers - i)
            with torch.cuda.stream(stream_load):
                m_next = Module(model_instance.model.layers, i, i + l_next).to('cuda', non_blocking=True)

        # Wait for the current chunk to finish loading before computing
        stream_compute.wait_stream(stream_load)
        
        # Compute on the current chunk
        with torch.cuda.stream(stream_compute):
            with torch.no_grad():
                x = m_current(x, attention_mask)
        
        # Offload the previously used chunk (if it exists)
        if m_to_offload:
            # Wait for compute to finish before offloading to prevent race condition
            stream_offload.wait_stream(stream_compute)
            with torch.cuda.stream(stream_offload):
                m_to_offload.to('cpu', non_blocking=True)

        # Advance the pipeline
        m_to_offload = m_current
        m_current = m_next
        i += layers_per_chunk

    # Handle the final cleanup outside the loop
    # stream_compute.synchronize()
    # stream_load.synchronize()
    # stream_offload.synchronize()
    
    # Ensure the last computed module is offloaded as well
    if m_to_offload:
        m_to_offload.to('cpu')
    if m_current: # Should be None but for safety
        m_current.to('cpu')
    if m_next:
        m_next.to('cpu')
    
    del m_to_offload, m_current, m_next
    
    torch.cuda.empty_cache()
    return x


def model_inference(question, tokens=250):
    start_total_time = time.time()
    
    model.to('cpu')
    
    prompt = tok(question, return_tensors='pt').to('cpu')
    input_ids = prompt['input_ids'].to('cuda')
    # Correctly move attention mask to CUDA and convert its dtype
    attention_mask = prompt['attention_mask'].to('cuda')
    del prompt

    # Move static parts of the model to GPU once
    model.model.embed_tokens.to('cuda')
    model.model.norm.to('cuda')
    model.lm_head.to('cuda')
    rotary_emb = model.model.rotary_emb
    
    generated_text_tokens = [] 

    for k in range(tokens):
        x = model.model.embed_tokens(input_ids)
        
        seq_len = x.shape[1]
        batch_size = x.shape[0]
        position_ids = torch.arange(seq_len, dtype=torch.long, device=x.device).unsqueeze(0).expand(batch_size, -1)
        cos, sin = rotary_emb(x=x,position_ids=position_ids)

        # Create a 4D attention mask with the correct dtype
        causal_mask = torch.triu(torch.ones((1, seq_len, seq_len), device=x.device, dtype=torch.bool), diagonal=1)
        causal_mask = ~causal_mask
        causal_mask = causal_mask.view(1, 1, seq_len, seq_len)
        causal_mask = causal_mask * attention_mask.view(1, 1, 1, seq_len)
        causal_mask = (causal_mask == 0).float() * torch.finfo(x.dtype).min
        
        with torch.no_grad():
            # Forcing layers_per_chunk to 5 as requested to test pipelining
            layers_per_chunk = 5
            x = compute(model, x, layers_per_chunk, cos, sin, k, tokens, causal_mask)
            
            x = model.model.norm(x)
            x = model.lm_head(x)
        
        # Reverting to the original greedy decoding using argmax as requested
        x1 = x[:,-1,:]
        next_token = torch.argmax(x1, dim=-1, keepdim=True)
        
        print(tok.decode(next_token[0]), end="")
        generated_text_tokens.append(tok.decode(next_token[0]))
        
        input_ids = torch.cat([input_ids, next_token.to('cuda')], dim=-1)
        attention_mask = torch.cat([attention_mask, torch.ones(attention_mask.shape[0], 1).to('cuda')], dim=-1)
    
    # Final cleanup at the very end
    model.model.embed_tokens.to('cpu')
    model.model.norm.to('cpu')
    model.lm_head.to('cpu')
    torch.cuda.empty_cache() 
    
    end_total_time = time.time()
    print(f"\nTime taken for {tokens} tokens: {end_total_time - start_total_time:.2f} seconds")
    return "".join(generated_text_tokens)

In [18]:
print("Starting inference...")
start = time.time()
model_inference("HI", 10)
print()
print("Total time", time.time() - start)

Starting inference...


TypeError: Module.forward() missing 2 required positional arguments: 'sin' and 'attention_mask'

# 2 workers

In [ ]:
import torch
from transformers import AutoModelForCausalLM,AutoTokenizer,BitsAndBytesConfig
from torch import nn
import time
import gc
# Add at the beginning of your script:
torch.backends.cudnn.benchmark = True
torch.backends.cuda.matmul.allow_tf32 = True
bnb_config=BitsAndBytesConfig(
            load_in_8bit=True,
            # bnb_4bit_compute_dtype=torch.bfloat16,
            # bnb_4bit_use_double_quant=True,
            # bnb_8bit_quant_type='nf4',
        )
model=AutoModelForCausalLM.from_pretrained("meta-llama/Llama-3.2-1B",
                                           torch_dtype=torch.bfloat16,
                                           use_cache=True,
                                        #    quantization_config=bnb_config
                                           )
# print(model)
tok=AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-1B")

/home/pa/miniconda3/envs/gen_ai/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
def get_layers(model,drop=2):
    device_memory=(torch.cuda.get_device_properties('cuda').total_memory)/(1024**3)
    l=model.model.layers[0]
    num_params = sum(p.numel() for p in l.parameters())
    param_size_bytes = num_params * 2
    param_size_GB = param_size_bytes / (1024 ** 3)
    return int(device_memory/param_size_GB)-drop

In [ ]:
get_layers(model)


65

In [ ]:
print(len(model.model.layers))

16


In [ ]:
class Module(nn.Module):
    def __init__(self, layers, i, j):
        super().__init__()
        self.model = nn.ModuleList(layers[i:j])
        
    @torch.compile
    def forward(self, x, cos, sin,attention_mask):
        for layer in self.model:
            x = layer(x, position_embeddings=(cos, sin))[0]
        return x
layers=model.model.layers


# Usage
# compiled_block = torch.compile(FusedBlock(layers[i:j]))

# def compute(model, x, layers_per_chunk, cos, sin,k,max_new_tokens,attention_mask):
#     total_layers = len(model.model.layers)
#     gpu_stream = torch.cuda.current_stream()
#     if layers_per_chunk >= total_layers:
#         # model = model.to('cuda')
#         # out = model({'inputs_embeds': x})['logits'][:, -1, :]
#         # return out.argmax(-1)
#         i=0
#         l=total_layers
#         m=Module(model.model.layers,i,i+l).to('cuda')
#         x=m(x.to('cuda', non_blocking=True),cos.to('cuda',non_blocking=True),sin.to('cuda',non_blocking=True),attention_mask.to('cuda'))
#         if(k==max_new_tokens-1):
#             m=m.to('cpu')
#             torch.cuda.empty_cache()
#         return x
#     i = 0
#     rem = total_layers
#     while rem > 0:
#         l = min(layers_per_chunk, rem)
        
#         m = Module(model.model.layers, i, i + l).to('cuda',non_blocking=True)
#         # torch.cuda.current_stream().wait_stream(gpu_stream)
#         # m=torch.compile(m)
#         # Run chunk
#         with torch.no_grad():
#             x=m(x.to('cuda', non_blocking=True),cos.to('cuda',non_blocking=True),sin.to('cuda',non_blocking=True),attention_mask.to('cuda'))
#         if(k==max_new_tokens-1):
#             m=m.to('cpu')
#             torch.cuda.empty_cache()
#         m=m.to('cpu')
#         torch.cuda.empty_cache()
#         # update indices
#         i += l
#         rem -= l
#     torch.cuda.current_stream().synchronize()
#     return x

In [ ]:
def compute(model_instance, x, layers_per_chunk, cos, sin, k, max_new_tokens, attention_mask):
    total_layers = len(model_instance.model.layers)
    stream_compute = torch.cuda.Stream()
    stream_transfer = torch.cuda.Stream()
    if layers_per_chunk >= total_layers:
        i = 0
        l = total_layers
        m = Module(model_instance.model.layers, i, i + l).to('cuda')
        x = m(x.to('cuda'), cos.to('cuda'), sin.to('cuda'), attention_mask.to('cuda'))
        if k == max_new_tokens - 1:
            m = m.to('cpu')
            torch.cuda.empty_cache()
        return x
    i=0
    rem = total_layers
    m_current = None
    m_next = None
    if rem > 0:
        l_first = min(layers_per_chunk, rem)
        with torch.cuda.stream(stream_transfer):
            m_next = Module(model_instance.model.layers, i, i + l_first).to('cuda', non_blocking=True)
        i += l_first
        rem -= l_first

    while True:
        stream_compute.wait_stream(stream_transfer)
        m_current = m_next
        m_next = None
        with torch.cuda.stream(stream_compute):
            with torch.no_grad():
                x = m_current(x.to('cuda', non_blocking=True), 
                              cos.to('cuda', non_blocking=True), 
                              sin.to('cuda', non_blocking=True), 
                              attention_mask.to('cuda', non_blocking=True))
        if rem == 0:
            stream_compute.synchronize()
            m_current = m_current.to('cpu')
            torch.cuda.empty_cache()
            break
        with torch.cuda.stream(stream_transfer):
            m_current = m_current.to('cpu', non_blocking=True)
            del m_current
            l_next = min(layers_per_chunk, rem)
            m_next = Module(model_instance.model.layers, i, i + l_next).to('cuda', non_blocking=True)
        torch.cuda.empty_cache() 
        i += l_next
        rem -= l_next
    return x

In [ ]:
def model_inference(question,tokens=250):
    prompt=tok(question,return_tensors='pt').to('cpu')
    input_ids=prompt['input_ids'].to('cuda',non_blocking=True)
    attention_mask=prompt['attention_mask']
    model.model.embed_tokens.to('cuda')
    model.model.norm.to('cuda') 
    model.lm_head.to('cuda')
    # x = model.model.embed_tokens(input_ids)
    rotary_emb = model.model.rotary_emb
    
    del prompt
    for k in range(tokens):
        torch.cuda.empty_cache()
        # model.model.embed_tokens.to('cuda')
        # if i!=0:
        x = model.model.embed_tokens(input_ids)
        seq_len = x.shape[1]
        batch_size = x.shape[0]
        position_ids = torch.arange(seq_len, dtype=torch.long, device=x.device).unsqueeze(0).expand(batch_size, -1)
        
        cos, sin = rotary_emb(x=x,position_ids=position_ids)
        # input_ids.to('cpu')
        # torch.cuda.empty_cache()
        # model.model.embed_tokens.to('cpu')
        # torch.cuda.empty_cache()
        
        cnt=0
        with torch.no_grad():
            l1=get_layers(model,7)
            x=compute(model,x,1,cos,sin,k,tokens,attention_mask)
            # model.model.norm.to('cuda')
            x=model.model.norm(x)
            # model.model.norm.to('cpu')
            # torch.cuda.empty_cache()
            # model.lm_head.to('cuda')
            x=model.lm_head(x)
            # model.lm_head.to('cpu')
            # torch.cuda.empty_cache()
        
        x1=x[:,-1,:]
        x1=torch.argmax(x1,dim=-1,keepdim=True)
        print(tok.decode(x1[0]),end="")
        input_ids=torch.cat([input_ids,x1],dim=-1) 
        # x1=x.argmax(-1)
        # print(tok.decode(x1[0]),end="\n")
        # question=question+" "+tok.decode(x1[0])
        

In [ ]:
start=time.time()
model_inference("HI",10)
print()
print("Total time",time.time()-start)

, I am a 19 year old girl who
Total time 4.548299551010132


# other

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from torch import nn
import time
import gc

# Add at the beginning of your script:
torch.backends.cudnn.benchmark = True
torch.backends.cuda.matmul.allow_tf32 = True

# Load the model and tokenizer
model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-3.2-1B",
                                             torch_dtype=torch.bfloat16,
                                             use_cache=True,
                                             )
model.to('cpu')

tok = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-1B")


/home/pa/miniconda3/envs/gen_ai/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:


def get_layers(model, drop=2):
    """
    Calculates the maximum number of layers that can fit on the GPU.
    This function has been improved to account for static model parts
    and provide a more robust chunk size calculation.

    Args:
        model: The model instance.
        drop (int): A memory buffer in GB to leave on the GPU.

    Returns:
        int: The number of layers per chunk.
    """
    device_memory = (torch.cuda.get_device_properties('cuda').total_memory) / (1024**3)
    l = model.model.layers[0]
    num_params = sum(p.numel() for p in l.parameters())
    param_size_bytes = num_params * 2 # Using bfloat16 for correct size calculation
    param_size_GB = param_size_bytes / (1024 ** 3)
    
    # Adjusting for static model parts which are loaded once
    static_mem_gb = 0
    if hasattr(model.model, 'embed_tokens'):
        static_mem_gb += sum(p.numel() for p in model.model.embed_tokens.parameters()) * 2 / (1024**3)
    if hasattr(model.model, 'norm'):
        static_mem_gb += sum(p.numel() for p in model.model.norm.parameters()) * 2 / (1024**3)
    if hasattr(model, 'lm_head'):
        static_mem_gb += sum(p.numel() for p in model.lm_head.parameters()) * 2 / (1024**3)

    available_gpu_memory_for_layers_gb = device_memory - drop - static_mem_gb
    if available_gpu_memory_for_layers_gb <= 0:
        print(f"Warning: Not enough GPU memory. Available: {device_memory:.2f} GB. Falling back to 1 layer per chunk.")
        return 1
    
    if param_size_GB == 0:
        return 1

    return max(1, int(available_gpu_memory_for_layers_gb / param_size_GB))


class Module(nn.Module):
    """A wrapper for a chunk of layers."""
    def __init__(self, layers, i, j):
        super().__init__()
        self.model = nn.ModuleList(layers[i:j])
    
    @torch.compile
    def forward(self, x, cos, sin, attention_mask):
        for layer in self.model:
            # Pass the attention mask to the layer
            x = layer(x, position_embeddings=(cos, sin), attention_mask=attention_mask)[0]
        return x


def compute(model_instance, x, layers_per_chunk, cos, sin, k, max_new_tokens, attention_mask):
    """
    Computes a forward pass using a three-worker pipeline with three explicit streams.
    Worker 1: Loading new layers (stream_load)
    Worker 2: Computation (stream_compute)
    Worker 3: Offloading old layers (stream_offload)
    """
    total_layers = len(model_instance.model.layers)

    stream_compute = torch.cuda.Stream()
    stream_load = torch.cuda.Stream()
    stream_offload = torch.cuda.Stream()

    # Case 1: All layers fit on the GPU, no offloading needed.
    if layers_per_chunk >= total_layers:
        m = Module(model_instance.model.layers, 0, total_layers).to('cuda')
        x = m(x.to('cuda'), cos.to('cuda'), sin.to('cuda'), attention_mask.to('cuda'))
        if k == max_new_tokens - 1:
            m.to('cpu')
            torch.cuda.empty_cache()
            del m
        return x

    # Case 2: Pipelined offloading with a robust, explicit three-stage loop.
    i = 0
    m_to_offload = None # This will hold the chunk from the previous iteration to be offloaded
    m_current = None # This is the chunk currently being computed
    
    # Priming the pipeline: Load the first chunk
    with torch.cuda.stream(stream_load):
        m_current = Module(model_instance.model.layers, i, i + layers_per_chunk).to('cuda', non_blocking=True)
    i += layers_per_chunk
    
    while i <= total_layers + layers_per_chunk: # Loop until all layers are processed and the last chunk is offloaded
        # 1. Load the next chunk on a separate stream if there are layers left
        m_next = None
        if i < total_layers:
            l_next = min(layers_per_chunk, total_layers - i)
            with torch.cuda.stream(stream_load):
                m_next = Module(model_instance.model.layers, i, i + l_next).to('cuda', non_blocking=True)
        
        # 2. Wait for the current chunk to finish loading, then compute
        stream_compute.wait_stream(stream_load)
        with torch.cuda.stream(stream_compute):
            with torch.no_grad():
                x = m_current(x, cos, sin, attention_mask)
        
        # 3. Wait for the computation to finish, then offload the previous chunk
        if m_to_offload:
            stream_offload.wait_stream(stream_compute)
            with torch.cuda.stream(stream_offload):
                m_to_offload.to('cpu', non_blocking=True)
        
        # Advance the pipeline to the next stage
        m_to_offload = m_current
        m_current = m_next
        i += layers_per_chunk

    # Final cleanup after the loop
    stream_compute.synchronize()
    stream_load.synchronize()
    stream_offload.synchronize()
    
    del m_to_offload, m_current, m_next
    
    torch.cuda.empty_cache()
    return x


def model_inference(question, tokens=250):
    start_total_time = time.time()
    
    model.to('cpu')
    
    prompt = tok(question, return_tensors='pt').to('cpu')
    input_ids = prompt['input_ids'].to('cuda')
    # Correctly move attention mask to CUDA and convert its dtype
    attention_mask = prompt['attention_mask'].to('cuda')
    del prompt

    # Move static parts of the model to GPU once
    model.model.embed_tokens.to('cuda')
    model.model.norm.to('cuda')
    model.lm_head.to('cuda')
    rotary_emb = model.model.rotary_emb
    
    generated_text_tokens = [] 

    for k in range(tokens):
        x = model.model.embed_tokens(input_ids)
        
        seq_len = x.shape[1]
        batch_size = x.shape[0]
        position_ids = torch.arange(seq_len, dtype=torch.long, device=x.device).unsqueeze(0).expand(batch_size, -1)
        cos, sin = rotary_emb(x=x,position_ids=position_ids)

        # Create a 4D attention mask with the correct dtype
        causal_mask = torch.triu(torch.ones((1, seq_len, seq_len), device=x.device, dtype=torch.bool), diagonal=1)
        causal_mask = ~causal_mask
        causal_mask = causal_mask.view(1, 1, seq_len, seq_len)
        causal_mask = causal_mask * attention_mask.view(1, 1, 1, seq_len)
        causal_mask = (causal_mask == 0).float() * torch.finfo(x.dtype).min
        
        with torch.no_grad():
            # Forcing layers_per_chunk to 5 as requested to test pipelining
            layers_per_chunk = 5
            x = compute(model, x, layers_per_chunk, cos, sin, k, tokens, causal_mask)
            
            x = model.model.norm(x)
            x = model.lm_head(x)
        
        # Reverting to the original greedy decoding using argmax as requested
        x1 = x[:,-1,:]
        next_token = torch.argmax(x1, dim=-1, keepdim=True)
        
        print(tok.decode(next_token[0]), end="")
        generated_text_tokens.append(tok.decode(next_token[0]))
        
        input_ids = torch.cat([input_ids, next_token.to('cuda')], dim=-1)
        attention_mask = torch.cat([attention_mask, torch.ones(attention_mask.shape[0], 1).to('cuda')], dim=-1)
    
    # Final cleanup at the very end
    model.model.embed_tokens.to('cpu')
    model.model.norm.to('cpu')
    model.lm_head.to('cpu')
    torch.cuda.empty_cache() 
    
    end_total_time = time.time()
    print(f"\nTime taken for {tokens} tokens: {end_total_time - start_total_time:.2f} seconds")
    return "".join(generated_text_tokens)




In [ ]:

start=time.time()
model_inference("HI",10)
print()
print("Total time",time.time()-start)

birbirbirbirbirbirbirbirbirbir
Time taken for 10 tokens: 4.10 seconds

Total time 4.095994472503662


12.1
11.9
11.5
11.9
10.9
11.1
10.3
9.5
9.0 (10)
9.3 (11)
10.5 (12)
9.8 (13)
9.9 (14)
9.1 (15)

1.9 (16) 

In [ ]:
question="hi how are "
tok=AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-1B")
prompt=tok(question,return_tensors='pt').to('cuda')
input_ids=prompt['input_ids']
model.model.embed_tokens.to('cuda')
x = model.model.embed_tokens(input_ids)
input_ids.to('cpu')
torch.cuda.empty_cache()
model.model.embed_tokens.to('cpu')
torch.cuda.empty_cache()
seq_len = x.shape[1]
batch_size = x.shape[0]
position_ids = torch.arange(seq_len, dtype=torch.long, device=x.device).unsqueeze(0).expand(batch_size, -1)
rotary_emb = model.model.rotary_emb
cos, sin = rotary_emb(x=x,position_ids=position_ids)

In [ ]:
class module(nn.Module):
    def __init__(self,layers,i,j):
        super().__init__()
        self.model=nn.ModuleList(layers[i:j+1])
    def forward(self,x,cos,sin):
        for l in self.model:
            x=l(x,position_embeddings=(cos,sin))[0]
        return x

In [ ]:
import torch.nn as nn
class Module(nn.Module):
    def __init__(self, layers, i, j):
        super().__init__()
        self.model = nn.ModuleList(layers[i:j])

    def forward(self, x, cos, sin):
        for layer in self.model:
            x = layer(x, position_embeddings=(cos, sin))[0]
        return x

def compute(model, x, layers_per_chunk, cos, sin):
    total_layers = len(model.model.layers)
    if layers_per_chunk >= total_layers:
        model = model.to('cuda')
        out = model({'inputs_embeds': x})['logits'][:, -1, :]
        return out.argmax(-1)
    i = 0
    rem = total_layers
    while rem > 0:
        l = min(layers_per_chunk, rem)
        m = Module(model.model.layers, i, i + l)

        # Run chunk
        x = m(x.to('cpu'), cos.to('cpu'), sin.to('cpu'))

        # update indices
        i += l
        rem -= l

    return x


In [ ]:
y=compute(model,x,4,cos,sin)
y
model.model.norm.to('cpu')
x=model.model.norm(y)
model.model.norm.to('cpu')
torch.cuda.empty_cache()
model.lm_head.to('cpu')
x=model.lm_head(x)
# tok.decode(x)
x

tensor([[[ 7.0544,  9.0268, 13.3233,  ..., -3.7595, -3.7596, -3.7596],
         [17.6009,  9.2103,  9.4629,  ..., -0.6160, -0.6167, -0.6170],
         [12.9647,  9.7460,  8.3662,  ..., -0.2670, -0.2677, -0.2679],
         [14.1844, 10.6885,  6.5441,  ...,  0.4012,  0.4009,  0.4002],
         [ 7.1437,  4.0489,  5.2485,  ...,  1.8494,  1.8495,  1.8485]]],
       grad_fn=<UnsafeViewBackward0>)

In [ ]:
cos.device

device(type='cuda', index=0)

In [ ]:
m=module(layers,0,7)
m(x.to('cpu'),cos.to('cpu'),sin.to('cpu'))

tensor([[[ 0.2278, -0.1952,  1.0512,  ...,  0.4470,  0.9027,  0.4017],
         [-0.0171,  0.0437,  0.0796,  ..., -0.0353, -0.0933, -0.0494],
         [-0.0076,  0.0305, -0.0894,  ..., -0.1101, -0.0858,  0.0215],
         [-0.0955,  0.1930, -0.0288,  ..., -0.0590, -0.0255,  0.1503],
         [-0.0718,  0.0508, -0.0462,  ...,  0.0025, -0.0664, -0.0365]]],
       grad_fn=<AddBackward0>)

In [ ]:
question="hi how are "
tok=AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-1B")
prompt=tok(question,return_tensors='pt').to('cuda')
input_ids=prompt['input_ids']
model.model.embed_tokens.to('cuda')
x = model.model.embed_tokens(input_ids)
input_ids.to('cpu')
torch.cuda.empty_cache()
model.model.embed_tokens.to('cpu')
torch.cuda.empty_cache()
seq_len = x.shape[1]
batch_size = x.shape[0]
position_ids = torch.arange(seq_len, dtype=torch.long, device=x.device).unsqueeze(0).expand(batch_size, -1)
rotary_emb = model.model.rotary_emb
cos, sin = rotary_emb(x=x,position_ids=position_ids)

In [ ]:
class Module(nn.Module):
    def __init__(self, layers, i, j):
        super().__init__()
        self.model = nn.ModuleList(layers[i:j])

    def forward(self, x, cos, sin):
        for layer in self.model:
            x = layer(x, position_embeddings=(cos, sin))[0]
        return x
def compute(model, x, layers_per_chunk, cos, sin):
    total_layers = len(model.model.layers)
    if layers_per_chunk >= total_layers:
        # model = model.to('cuda')
        # out = model({'inputs_embeds': x})['logits'][:, -1, :]
        # return out.argmax(-1)
        i=0
        l=total_layers
        m=Module(model.model.layers,i,i+l).to('cuda')
        x=m(x.to('cuda'),cos.to('cuda'),sin.to('cuda'))
        m=m.to('cpu')
        torch.cuda.empty_cache()
        return x
    i = 0
    rem = total_layers
    while rem > 0:
        l = min(layers_per_chunk, rem)
        m = Module(model.model.layers, i, i + l).to('cuda')

        # Run chunk
        x = m(x.to('cuda'), cos.to('cuda'), sin.to('cuda'))
        m=m.to('cpu')
        torch.cuda.empty_cache()
        # update indices
        i += l
        rem -= l

    return x

In [ ]:
y=compute(model,x,get_layers(model),cos,sin)
model.model.norm.to('cpu')
x=model.model.norm(y)
model.model.norm.to('cpu')
torch.cuda.empty_cache()
model.lm_head.to('cpu')
x=model.lm_head(x)
x

tensor([[[ 7.0544,  9.0268, 13.3233,  ..., -3.7595, -3.7596, -3.7596],
         [17.6009,  9.2103,  9.4629,  ..., -0.6160, -0.6167, -0.6170],
         [12.9647,  9.7460,  8.3662,  ..., -0.2670, -0.2677, -0.2679],
         [14.1844, 10.6885,  6.5441,  ...,  0.4012,  0.4009,  0.4002],
         [ 7.1437,  4.0489,  5.2485,  ...,  1.8494,  1.8495,  1.8485]]],
       grad_fn=<UnsafeViewBackward0>)

In [ ]:
model(input_ids.to('cpu'))

CausalLMOutputWithPast(loss=None, logits=tensor([[[ 7.0544,  9.0268, 13.3233,  ..., -3.7595, -3.7596, -3.7596],
         [17.6009,  9.2103,  9.4629,  ..., -0.6160, -0.6167, -0.6170],
         [12.9647,  9.7460,  8.3662,  ..., -0.2670, -0.2677, -0.2679],
         [14.1844, 10.6885,  6.5441,  ...,  0.4011,  0.4009,  0.4002],
         [ 7.1437,  4.0489,  5.2485,  ...,  1.8494,  1.8495,  1.8485]]],
       grad_fn=<UnsafeViewBackward0>), past_key_values=<transformers.cache_utils.DynamicCache object at 0x71d5247463e0>, hidden_states=None, attentions=None)

In [ ]:
y=compute(model,x,4,cos,sin)
y

tensor([[[ 0.4072, -0.0109,  0.1410,  ...,  0.2553, -0.2387, -0.3508],
         [ 0.1307, -0.0781,  0.1387,  ...,  0.4531,  0.1098, -0.2780],
         [ 0.1409,  0.0029,  0.2776,  ...,  0.8693,  0.6018, -0.1494],
         [-0.0449,  0.0310,  0.0482,  ...,  0.3607, -0.3491, -0.5668],
         [-0.0326,  0.3774, -0.2075,  ...,  0.3641, -0.2429, -0.2585]]],
       grad_fn=<AddBackward0>)

In [ ]:
model.model.norm.to('cpu')
x=model.model.norm(y)
model.model.norm.to('cpu')
torch.cuda.empty_cache()
model.lm_head.to('cpu')
x=model.lm_head(x)
# tok.decode(x)
x

tensor([[[ 0.4545,  2.3900, -1.2563,  ..., -1.2801, -1.2815, -1.2818],
         [-0.5335,  1.2898, -0.0679,  ..., -0.9111, -0.9123, -0.9122],
         [ 0.2582,  0.9092,  1.3728,  ..., -1.5733, -1.5758, -1.5756],
         [-1.0597,  2.9733,  0.6937,  ..., -0.5993, -0.6006, -0.6005],
         [ 0.2114,  0.6242,  1.3618,  ..., -0.0651, -0.0660, -0.0650]]],
       grad_fn=<UnsafeViewBackward0>)

In [ ]:
i=0
j=3
l=4
rem=16
while(rem!=0):
    if(rem>=l):
        m=nn.Sequential(
            layers[i:j]
        )
        print(i," ",j)
        rem-=l
        i+=l;j+=l
    else:
        m=nn.Sequential(
            layers[i:i+rem-1]
        )
        print('else')
        print(i," ",i+rem-1)
        rem=0


0   3
4   7
8   11
12   15


In [ ]:
m=nn.Sequential(
        layers[0:3]
)
m

Sequential(
  (0): ModuleList(
    (0-2): 3 x LlamaDecoderLayer(
      (self_attn): LlamaAttention(
        (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
        (k_proj): Linear(in_features=2048, out_features=512, bias=False)
        (v_proj): Linear(in_features=2048, out_features=512, bias=False)
        (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
      )
      (mlp): LlamaMLP(
        (gate_proj): Linear(in_features=2048, out_features=8192, bias=False)
        (up_proj): Linear(in_features=2048, out_features=8192, bias=False)
        (down_proj): Linear(in_features=8192, out_features=2048, bias=False)
        (act_fn): SiLU()
      )
      (input_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
      (post_attention_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
    )
  )
)

if int((torch.cuda.get_device_properties('cuda').total_memory)/(1024**3)/param_size_MB)-2 > total layers  : load whole model into sequential model

else load int((torch.cuda.get_device_properties('cuda').total_memory)/(1024**3)/param_size_MB)-2 layers into the seq 

keep track of the layers which went inside the seq model

[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15] if 6-2=4 layers are allowed then iterations = while i not>= number of layers 

In [ ]:
# LlamaRMSNorm((2048,), eps=1e-05)
#     (rotary_emb): LlamaRotaryEmbedding()
# print((sum(p.numel() for p in model.model.norm.parameters())*2)/(1024**3))
# print((sum(p.numel() for p in model.lm_head.parameters())*2)/(1024**3))
# print((sum(p.numel() for p in model.model.embed_tokens.parameters())*2)/(1024**3))


3.814697265625e-06
0.4892578125
0.4892578125


In [ ]:
model=model.to('cpu')

In [ ]:
torch.cuda.empty_cache()

In [ ]:
question=input()
converstation=[
    {
        "role":"system",
        "content":"You are an assitant help user by solving their query"
    },
    {
        "role":"user",
        "content":f"query-{question}"
    }
]

In [ ]:
question="hi how are "
tok=AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-1B")
# prompt=tok.apply_chat_template(converstation)
prompt=tok(question,return_tensors='pt').to('cuda')
# prompt.to('cpu'//)


In [ ]:
prompt

{'input_ids': tensor([[128000,   6151,   1268,    527,    220]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1]], device='cuda:0')}

In [ ]:
# out=model.generate(**prompt,max_new_tokens=1000)
# print(tok.decode(out[0]))

In [ ]:
prompt['input_ids'].device

device(type='cuda', index=0)

In [ ]:
class module(nn.Module):
    def __init__(self,layers,i,j):
        super().__init__()
        self.model=layers[i:j]
    def forward(self,x,cos,sin):
        for l in self.model:
            x=l(x,position_embeddings=(cos,sin))[0]
            print()
        return x

In [ ]:
m=module(layers,0,3)
m(x.to('cpu'),cos.to('cpu'),sin.to('cpu'))

tensor([[[ 0.2181, -0.3543,  1.0459,  ...,  0.3998,  0.9639,  0.3820],
         [ 0.0501, -0.0546, -0.0243,  ...,  0.1164, -0.1678,  0.0384],
         [ 0.0353, -0.0942, -0.1205,  ..., -0.0287, -0.1063,  0.0035],
         [ 0.0649,  0.0549, -0.0413,  ..., -0.0517, -0.0086,  0.0737],
         [-0.0263,  0.0541, -0.0978,  ...,  0.0248, -0.0258, -0.0238]]],
       grad_fn=<AddBackward0>)

In [ ]:
prompt['input_ids']

NameError: name 'prompt' is not defined

tensor([[[ 2.8381e-03,  3.3264e-03, -9.8877e-03,  ..., -1.7700e-03,
           8.4305e-04,  7.0190e-04],
         [-4.0039e-02,  2.8320e-02,  2.8076e-02,  ...,  1.8311e-02,
           9.5215e-03,  4.7363e-02],
         [-7.2937e-03,  4.3335e-03,  3.0029e-02,  ...,  8.0109e-05,
          -4.8828e-03,  1.2451e-02],
         [-1.6785e-03,  1.7334e-02,  3.9551e-02,  ..., -1.1047e-02,
          -1.5869e-02,  1.1520e-03],
         [ 5.0964e-03, -2.4902e-02,  3.6133e-02,  ..., -1.6602e-02,
          -1.8677e-02, -3.5156e-02]]], grad_fn=<ToCopyBackward0>)
tensor([[[ 1.0000,  1.0000,  1.0000,  1.0000,  1.0000,  1.0000,  1.0000,
           1.0000,  1.0000,  1.0000,  1.0000,  1.0000,  1.0000,  1.0000,
           1.0000,  1.0000,  1.0000,  1.0000,  1.0000,  1.0000,  1.0000,
           1.0000,  1.0000,  1.0000,  1.0000,  1.0000,  1.0000,  1.0000,
           1.0000,  1.0000,  1.0000,  1.0000,  1.0000,  1.0000,  1.0000,
           1.0000,  1.0000,  1.0000,  1.0000,  1.0000,  1.0000,  1.0000,
        

AttributeError: 'tuple' object has no attribute 'dtype'

In [ ]:
m=model.model.layers[0]
m(x.to('cpu'),position_embeddings=(cos.to('cpu'),sin.to('cpu')))[0]

tensor([[[ 0.0131, -0.0472,  0.0412,  ..., -0.0090,  0.0760, -0.0241],
         [-0.0173, -0.0174, -0.0555,  ...,  0.0072, -0.0251, -0.0046],
         [-0.0093,  0.0029, -0.1033,  ..., -0.0247, -0.0518,  0.0019],
         [ 0.0227,  0.0492, -0.0206,  ..., -0.0301,  0.0070,  0.0488],
         [-0.0341,  0.0040, -0.0712,  ..., -0.0127, -0.0116,  0.0161]]],
       grad_fn=<AddBackward0>)

In [ ]:
cos.device

device(type='cuda', index=0)

In [ ]:
# out=model(prompt['input_ids'].to('cpu'))
m=layers[:3]


ModuleList(
  (0-2): 3 x LlamaDecoderLayer(
    (self_attn): LlamaAttention(
      (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
      (k_proj): Linear(in_features=2048, out_features=512, bias=False)
      (v_proj): Linear(in_features=2048, out_features=512, bias=False)
      (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
    )
    (mlp): LlamaMLP(
      (gate_proj): Linear(in_features=2048, out_features=8192, bias=False)
      (up_proj): Linear(in_features=2048, out_features=8192, bias=False)
      (down_proj): Linear(in_features=8192, out_features=2048, bias=False)
      (act_fn): SiLU()
    )
    (input_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
    (post_attention_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
  )
)

In [ ]:
x

tensor([[[ 2.8381e-03,  3.3264e-03, -9.8877e-03,  ..., -1.7700e-03,
           8.4305e-04,  7.0190e-04],
         [-4.0039e-02,  2.8320e-02,  2.8076e-02,  ...,  1.8311e-02,
           9.5215e-03,  4.7363e-02],
         [-7.2937e-03,  4.3335e-03,  3.0029e-02,  ...,  8.0109e-05,
          -4.8828e-03,  1.2451e-02],
         [-1.6785e-03,  1.7334e-02,  3.9551e-02,  ..., -1.1047e-02,
          -1.5869e-02,  1.1520e-03],
         [ 5.0964e-03, -2.4902e-02,  3.6133e-02,  ..., -1.6602e-02,
          -1.8677e-02, -3.5156e-02]]], device='cuda:0',
       grad_fn=<EmbeddingBackward0>)

In [ ]:
model(prompt['input_ids'].to('cpu'))

CausalLMOutputWithPast(loss=None, logits=tensor([[[ 7.0544,  9.0268, 13.3233,  ..., -3.7595, -3.7596, -3.7596],
         [17.6009,  9.2103,  9.4629,  ..., -0.6160, -0.6167, -0.6170],
         [12.9647,  9.7460,  8.3662,  ..., -0.2670, -0.2677, -0.2679],
         [14.1844, 10.6885,  6.5441,  ...,  0.4011,  0.4009,  0.4002],
         [ 7.1437,  4.0489,  5.2485,  ...,  1.8494,  1.8495,  1.8485]]],
       grad_fn=<UnsafeViewBackward0>), past_key_values=<transformers.cache_utils.DynamicCache object at 0x71d526b7b8e0>, hidden_states=None, attentions=None)

In [ ]:
nextt=out['logits'][:,-1,:].argmax(dim=-1)
nextt
tok.decode(nextt)

'2'

In [ ]:
input_ids=prompt['input_ids']
del prompt
torch.cuda.empty_cache()

In [ ]:
input_ids.device

device(type='cuda', index=0)

In [ ]:
model.model.embed_tokens.to('cuda')
x = model.model.embed_tokens(input_ids)
input_ids.to('cpu')
# del input_ids
torch.cuda.empty_cache()
model.model.embed_tokens.to('cpu')
torch.cuda.empty_cache()


In [ ]:
x

tensor([[[ 2.8381e-03,  3.3264e-03, -9.8877e-03,  ..., -1.7700e-03,
           8.4305e-04,  7.0190e-04],
         [-4.0039e-02,  2.8320e-02,  2.8076e-02,  ...,  1.8311e-02,
           9.5215e-03,  4.7363e-02],
         [-7.2937e-03,  4.3335e-03,  3.0029e-02,  ...,  8.0109e-05,
          -4.8828e-03,  1.2451e-02],
         [-1.6785e-03,  1.7334e-02,  3.9551e-02,  ..., -1.1047e-02,
          -1.5869e-02,  1.1520e-03],
         [ 5.0964e-03, -2.4902e-02,  3.6133e-02,  ..., -1.6602e-02,
          -1.8677e-02, -3.5156e-02]]], device='cuda:0',
       grad_fn=<EmbeddingBackward0>)

In [ ]:
# model.model.decoder.embed_positions.to('cuda')
# x = x + model.model.decoder.embed_positions(torch.arange(x.shape[1]).unsqueeze(0).to('cuda'))
# model.model.decoder.embed_positions.to('cpu')
# # del model.model.decoder.embed_positions
# torch.cuda.empty_cache()

In [ ]:
print(f"Model's max_position_embeddings: {model.config.max_position_embeddings}")

Model's max_position_embeddings: 131072


In [ ]:
x.shape

torch.Size([1, 5, 2048])

In [ ]:
seq_len = x.shape[1]
batch_size = x.shape[0]
position_ids = torch.arange(seq_len, dtype=torch.long, device=x.device).unsqueeze(0).expand(batch_size, -1)
rotary_emb = model.model.rotary_emb
cos, sin = rotary_emb(x=x,position_ids=position_ids)

In [1]:
from transformers import AutoTokenizer,AutoModelForCausalLM
import torch
import time
tokenizer=AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-1B")
model=AutoModelForCausalLM.from_pretrained('meta-llama/Llama-3.2-1B',torch_dtype=torch.bfloat16)

/home/pa/miniconda3/envs/gen_ai/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
question="hi how are "
prompt=tokenizer(question,return_tensors='pt').to('cuda')
input_ids=prompt['input_ids']
model.model.embed_tokens.to('cuda')
x = model.model.embed_tokens(input_ids)
input_ids.to('cpu')
torch.cuda.empty_cache()
model.model.embed_tokens.to('cpu')
torch.cuda.empty_cache()
seq_len = x.shape[1]
batch_size = x.shape[0]
position_ids = torch.arange(seq_len, dtype=torch.long, device=x.device).unsqueeze(0).expand(batch_size, -1)
rotary_emb = model.model.rotary_emb.to(x.device)
cos, sin = rotary_emb(x=x,position_ids=position_ids)

In [3]:
import torch
import time
from transformers.models.llama.modeling_llama import DynamicCache, Cache  # Correct imports

# === KV Extractor ===
def extract_and_reshape_kv(layer, x):
    """Extract KV using your approach and reshape for cache usage"""
    attn = layer.self_attn
    
    # Your approach: Extract raw projections
    k_raw = attn.k_proj(x).detach()  # [batch, seq_len, 512]
    v_raw = attn.v_proj(x).detach()  # [batch, seq_len, 512]
    
    # Convert to cache format
    bsz, seq_len, _ = x.shape
    num_kv_heads = attn.num_key_value_heads  # 8 for your model
    head_dim = attn.head_dim  # 64 for your model
    
    # Reshape for multi-head attention cache format
    k_cache = k_raw.view(bsz, seq_len, num_kv_heads, head_dim).transpose(1, 2)  # [1, 8, seq_len, 64]
    v_cache = v_raw.view(bsz, seq_len, num_kv_heads, head_dim).transpose(1, 2)  # [1, 8, seq_len, 64]
    
    return (k_cache, v_cache)



# === Step 1: Initial Setup ===


# === Step 2: Initialize DynamicCache ===
# dynamic_cache = DynamicCache.with_empty_cache(model=model.model)
dynamic_cache = DynamicCache()

print("=== Initial KV Extraction Using DynamicCache ===")
model.model.rotary_emb.to('cuda')
start = time.time()
for _ in range(5):
    model.model.embed_tokens.to('cuda')
    # initial_input_ids = torch.tensor([[tokenizer.bos_token_id]], device='cuda')
    x = model.model.embed_tokens(input_ids)
    seq_len = x.shape[1]
    batch_size = x.shape[0]

    position_ids = torch.arange(seq_len, dtype=torch.long, device=x.device).unsqueeze(0).expand(batch_size, -1)
    rotary_emb = model.model.rotary_emb
    cos, sin = rotary_emb(x=x, position_ids=position_ids)
    for layer_idx, layer in enumerate(model.model.layers):
        # print(f"--- Layer {layer_idx + 1} ---")
        
        layer.to('cuda')
        x = x.to('cuda')

        with torch.no_grad():
            # Extract KV
            k_cache, v_cache = extract_and_reshape_kv(layer, x)

            # Use Cache wrapper to update DynamicCache
            dynamic_cache.update(
                k_cache.cuda(),
                v_cache.cuda(),
                layer_idx=layer_idx  # must match layer index
            )

            # Forward pass using dynamic cache
            out = layer(
                x,
                position_embeddings=(cos.to('cuda'), sin.to('cuda')),
                past_key_value=dynamic_cache,
                use_cache=True
            )
            x = out[0]

        layer.to('cpu')
        torch.cuda.empty_cache()
    with torch.no_grad():
        model.model.norm.to('cuda')
        x=model.model.norm(x)
        model.model.norm.to('cpu')
        torch.cuda.empty_cache()
        model.lm_head.to('cuda')
        x=model.lm_head(x)
        model.lm_head.to('cpu')
        torch.cuda.empty_cache()
        x1=x[:,-1,:]
        
        x1=torch.argmax(x1,dim=-1,keepdim=True)
        # input_ids=torch.cat([input_ids,x1],dim=-1)
        input_ids=x1
        # print(input_ids.shape)
        print(tokenizer.decode(x1[0]),end=" ")


print(f"\nInitial KV extraction time: {time.time() - start:.2f}s")
print(f"Final hidden state shape: {x.shape}")


=== Initial KV Extraction Using DynamicCache ===
лоб aca  wors  Ca  Ca 
Initial KV extraction time: 20.52s
Final hidden state shape: torch.Size([1, 1, 128256])


In [4]:
# seq_len = x.shape[1]
# batch_size = x.shape[0]
# position_ids = torch.arange(seq_len, dtype=torch.long, device=x.device).unsqueeze(0).expand(batch_size, -1)
# rotary_emb = model.model.rotary_emb
# cos, sin = rotary_emb(x=x,position_ids=position_ids)
# cnt=0
start=time.time()
for _ in range(5):
    model.model.embed_tokens.to('cuda')
    x = model.model.embed_tokens(input_ids)
    input_ids.to('cpu')
    torch.cuda.empty_cache()
    model.model.embed_tokens.to('cpu')
    torch.cuda.empty_cache()
    seq_len = x.shape[1]
    batch_size = x.shape[0]
    position_ids = torch.arange(seq_len, dtype=torch.long, device=x.device).unsqueeze(0).expand(batch_size, -1)
    rotary_emb = model.model.rotary_emb.to(x.device)
    cos, sin = rotary_emb(x=x,position_ids=position_ids)
    cnt=0
    for i in model.model.layers:
        # print(x.shape)
        # print(cos.shape)
        # print(sin.shape)
        cnt+=1
        # print(cnt)
        i.to('cuda')
        x.to('cuda')
        # print(x)
        with torch.no_grad():
            
            x=i(x,position_embeddings=(cos,sin),use_cache=True)
            # print(x)
            # print(x)
        i.to('cpu')
        torch.cuda.empty_cache()
        x=x[0]
        # x.to('cpu')
        torch.cuda.empty_cache()
            # time.sleep(3)
    with torch.no_grad():
        model.model.norm.to('cuda')
        x=model.model.norm(x)
        model.model.norm.to('cpu')
        torch.cuda.empty_cache()
        model.lm_head.to('cuda')
        x=model.lm_head(x)
        model.lm_head.to('cpu')
        torch.cuda.empty_cache()
        x1=x[:,-1,:]
        torch.cat([input_ids,x1],dim=-1)
        x1=torch.argmax(x1,dim=-1,keepdim=True)
        # print(tok.decode(x1[0]),end=" ")
print(time.time()-start)

24.809312343597412


In [4]:
x.shape

torch.Size([1, 5, 2048])

In [ ]:
x

tensor([[[ 4.7762, -5.0166, 13.8534,  ..., -1.9152,  1.8989, -3.0755],
         [-0.2314,  1.3026,  1.7797,  ..., -0.2708, -1.0364, -0.2083],
         [ 0.2276,  0.8936,  0.8521,  ..., -0.0246, -1.4944, -0.0458],
         [ 0.0748,  1.2204,  0.5945,  ..., -0.7305, -1.4607, -0.6370],
         [ 0.6602,  1.3918, -0.1689,  ..., -1.1154, -1.7489, -0.1802]]],
       device='cuda:0')

In [ ]:
x[:,-1,:]

tensor([[ 0.6602,  1.3918, -0.1689,  ..., -1.1154, -1.7489, -0.1802]],
       device='cuda:0')

In [5]:
with torch.no_grad():
    model.model.norm.to('cuda')
    x=model.model.norm(x)
    model.model.norm.to('cpu')
    torch.cuda.empty_cache()
    model.lm_head.to('cuda')
    x=model.lm_head(x)
    model.lm_head.to('cpu')
    torch.cuda.empty_cache()

In [ ]:
x

tensor([[[ 7.0544,  9.0268, 13.3233,  ..., -3.7595, -3.7596, -3.7596],
         [17.6009,  9.2103,  9.4629,  ..., -0.6160, -0.6167, -0.6170],
         [12.9647,  9.7460,  8.3662,  ..., -0.2670, -0.2677, -0.2679],
         [14.1844, 10.6885,  6.5441,  ...,  0.4011,  0.4009,  0.4002],
         [ 7.1437,  4.0489,  5.2485,  ...,  1.8494,  1.8495,  1.8485]]],
       device='cuda:0')

In [ ]:
prompt

NameError: name 'prompt' is not defined

In [ ]:
torch.cat([input_ids,x1],dim=-1)

tensor([[128000,   6151,   1268,    527,    220,     17]], device='cuda:0')

In [6]:
x1=x[:,-1,:]
x1=torch.argmax(x1,dim=-1,keepdim=True)
print(tok.decode(x1[0]),end=" ")
# print(x1)

2 

In [ ]:
# device=torch.device('cuda:0')
props=torch.cuda.get_device_properties('cuda')
props.total_memory/(1024**3)

7.6064453125

In [3]:
from transformers.models.llama.modeling_llama import DynamicCache
def model_inference(question, tokens=250):
    # --- Phase 1: Prefill the KV cache with the entire prompt ---
    prompt = tokenizer(question, return_tensors='pt').to('cuda')
    input_ids = prompt['input_ids']
    
    dynamic_cache = DynamicCache()
    
    with torch.no_grad():
        # Process the entire prompt at once to get the initial embeddings
        model.model.embed_tokens.to('cuda')
        x = model.model.embed_tokens(input_ids)
        
        seq_len = x.shape[1]
        batch_size = x.shape[0]
        position_ids = torch.arange(seq_len, dtype=torch.long, device=x.device).unsqueeze(0).expand(batch_size, -1)
        
        model.model.rotary_emb.to('cuda')
        cos, sin = model.model.rotary_emb(x=x, position_ids=position_ids)
        
        # This loop now correctly passes the cache to the layer and lets the layer update it
        for layer_idx, layer in enumerate(model.model.layers):
            layer.to('cuda')
            
            out = layer(
                x,
                position_embeddings=(cos, sin),
                past_key_value=dynamic_cache, # pass the cache
                use_cache=True               # enable caching
            )
            x = out[0]
            
            layer.to('cpu')
            torch.cuda.empty_cache()
    
    # Get the last hidden state and the last token ID from the prefill pass
    with torch.no_grad():
        model.model.norm.to('cuda')
        x = model.model.norm(x)
        model.model.norm.to('cpu')
        torch.cuda.empty_cache()
        
        model.lm_head.to('cuda')
        logits = model.lm_head(x)
        model.lm_head.to('cpu')
        torch.cuda.empty_cache()

    next_token_id = torch.argmax(logits[:, -1, :], dim=-1, keepdim=True)
    print(tokenizer.decode(input_ids[0]), end="")
    print(tokenizer.decode(next_token_id[0]), end="")
    
    # --- Phase 2: Autoregressive generation loop ---
    # We now pass only the single new token and the cache
    current_input_ids = next_token_id
    
    for _ in range(tokens):
        with torch.no_grad():
            # Get the new position ID, based on the current length of the cache
            current_position_id = torch.tensor([[dynamic_cache.get_seq_length()]], device='cuda')
            
            model.model.embed_tokens.to('cuda')
            x = model.model.embed_tokens(current_input_ids)
            
            model.model.rotary_emb.to('cuda')
            cos, sin = model.model.rotary_emb(x=x, position_ids=current_position_id)
            
            # Offloading loop for the single new token
            for layer_idx, layer in enumerate(model.model.layers):
                layer.to('cuda')
                
                out = layer(
                    x,
                    position_embeddings=(cos, sin),
                    past_key_value=dynamic_cache,
                    use_cache=True
                )
                x = out[0]
                
                layer.to('cpu')
                torch.cuda.empty_cache()
            
            # Final Norm and Head for the new token
            model.model.norm.to('cuda')
            x = model.model.norm(x)
            model.model.norm.to('cpu')
            torch.cuda.empty_cache()
            
            model.lm_head.to('cuda')
            logits = model.lm_head(x)
            model.lm_head.to('cpu')
            torch.cuda.empty_cache()
            
            next_token_id = torch.argmax(logits[:, -1, :], dim=-1, keepdim=True)
            print(tokenizer.decode(next_token_id[0]), end="")
            
            current_input_ids = next_token_id
        

In [4]:
dc=model_inference("Hi",10)

<|begin_of_text|>Hi, I'm a 20 year old girl who loves

In [5]:
layer = model.model.layers[0]
print(layer.self_attn.num_heads, layer.self_attn.head_dim)


AttributeError: 'LlamaAttention' object has no attribute 'num_heads'

In [35]:
print(dc[3][0].shape)

torch.Size([1, 8, 24, 64])


In [8]:
def model_inference(prompt_text, tokens=250):
    # Initial tokenization
    prompt = tok(prompt_text, return_tensors='pt')
    input_ids = prompt['input_ids'].to('cuda')  # [1, seq_len]
    del prompt
    torch.cuda.empty_cache()

    for _ in range(tokens):
        # Run only the latest token through embed layer and model
        model.model.embed_tokens.to('cuda')
        x = model.model.embed_tokens(input_ids)
        model.model.embed_tokens.to('cpu')
        torch.cuda.empty_cache()

        seq_len = x.shape[1]
        batch_size = x.shape[0]
        position_ids = torch.arange(seq_len, dtype=torch.long, device=x.device).unsqueeze(0).expand(batch_size, -1)

        rotary_emb = model.model.rotary_emb
        cos, sin = rotary_emb(x=x, position_ids=position_ids)

        # Transformer blocks
        for layer in model.model.layers:
            layer.to('cuda')
            x = x.to('cuda')
            with torch.no_grad():
                x = layer(x, position_embeddings=(cos, sin))
            layer.to('cpu')
            torch.cuda.empty_cache()
            x = x[0]  # Remove tuple

        # Final norm and lm_head
        with torch.no_grad():
            model.model.norm.to('cuda')
            x = model.model.norm(x)
            model.model.norm.to('cpu')
            torch.cuda.empty_cache()

            model.lm_head.to('cuda')
            logits = model.lm_head(x)
            model.lm_head.to('cpu')
            torch.cuda.empty_cache()

        next_token_logits = logits[:, -1, :]  # Only the last token matters
        next_token = torch.argmax(next_token_logits, dim=-1, keepdim=True)  # shape [1,1]

        # Append predicted token
        input_ids = torch.cat([input_ids, next_token], dim=-1)

        # Decode and print latest token
        print(tok.decode(next_token[0]), end="", flush=True)


In [10]:
start=time.time()
model_inference("Hi how are you ?",10)
print(time.time()-start)

 I am a new member here and I am very25.85783076286316


In [ ]:
def tokenize_with_oov(text, tokenizer, max_token_id=2048, oov_token='[OOV]'):
    # First, add [OOV] token to tokenizer if not present
    if oov_token not in tokenizer.get_vocab():
        tokenizer.add_special_tokens({'additional_special_tokens': [oov_token]})

    oov_id = tokenizer.convert_tokens_to_ids(oov_token)

    # Raw tokenization
    encoding = tokenizer(text, return_tensors='pt', add_special_tokens=False)
    input_ids = encoding['input_ids'][0]

    # Replace tokens ≥ max_token_id with OOV
    processed_ids = []
    for token_id in input_ids:
        if token_id >= max_token_id:
            processed_ids.append(oov_id)
        else:
            processed_ids.append(token_id)

    return torch.tensor([processed_ids]), encoding['attention_mask']

def model_inference(question, tokens=250):
    try:
        for _ in range(tokens):
            # prompt = tok(question, return_tensors='pt').to('cpu')
            # input_ids = prompt['input_ids']
            # attention_mask = prompt['attention_mask']
            input_ids, attention_mask = tokenize_with_oov(question, tok)

            del prompt
            torch.cuda.empty_cache()

            # Truncate if too long
            batch_size, seq_len = input_ids.shape
            max_positions = model.config.max_position_embeddings
            if seq_len > max_positions:
                input_ids = input_ids[:, -max_positions:]
                attention_mask = attention_mask[:, -max_positions:]
                seq_len = input_ids.shape[1]

            # Embeddings
            model.model.decoder.embed_tokens.to('cpu')
            x = model.model.decoder.embed_tokens(input_ids)

            # Positional IDs
            position_ids = torch.arange(seq_len, dtype=torch.long, device='cpu')
            position_ids = position_ids.unsqueeze(0).expand(batch_size, -1)

            model.model.decoder.embed_positions.to('cpu')
            x = x + model.model.decoder.embed_positions(position_ids)

            # Decoder layers
            for layer in model.model.decoder.layers:
                layer.to('cpu')
                x = x.to('cpu')
                with torch.no_grad():
                    x = layer(x, attention_mask=attention_mask.to(torch.float32))[0]
                layer.to('cpu')
                x = x.to('cpu')
                torch.cuda.empty_cache()

            # Final LM head
            model.lm_head.to('cpu')
            logits = model.lm_head(x.to('cpu'))
            model.lm_head.to('cpu')
            torch.cuda.empty_cache()

            # Decode
            next_token_id = logits.argmax(-1)
            next_token_text = tok.decode(next_token_id[0], skip_special_tokens=True)
            print(next_token_text, end=" ")
            question += " " + next_token_text

    except Exception as e:
        print("\n[Exception Caught]")
        print("Question So Far:", question)
        prompt = tok(question, return_tensors='pt').to('cpu')
        input_ids = prompt['input_ids']
        x = model.model.decoder.embed_tokens(input_ids)
        y = torch.arange(x.shape[1]).unsqueeze(0)
        print("Embedding Shape:", x[0].shape)
        print("Position ID Shape:", y.shape)
        print("Error:", e)


In [ ]:
model_inference("how are you 're post")

.''. you you'rewepost navigation .''. your you'reTheze is It.''..''.'re'reiresolicy姫conservancy how are you 're post .''. you you'rewepost navigation .''. your you'reTheze is It.''..''.'re'reiresolicy姫conservancy

torch.Size([35, 768])
torch.Size([1, 35])
index out of range in self


In [ ]:
import torch
import gc

def model_inference(question, tokens=250):
    # Ensure model components are initially on CPU
    model.model.decoder.embed_tokens.to('cpu')
    model.model.decoder.embed_positions.to('cpu')
    for layer in model.model.decoder.layers:
        layer.to('cpu')
    model.lm_head.to('cpu')
    
    max_position_embeddings = model.config.max_position_embeddings
    print(f"Model's max_position_embeddings: {max_position_embeddings}")

    # Initialize input_ids_for_next_step. This will be the tensor that grows.
    input_ids_for_next_step = tok(question, return_tensors='pt')['input_ids'] 
    
    gc.collect()
    torch.cuda.empty_cache()

    for _ in range(tokens):
        # The 'x' variable for the current iteration is the input_ids_for_next_step
        current_input_ids = input_ids_for_next_step 
        current_seq_len = current_input_ids.shape[1]

        # --- Truncation ---
        if current_seq_len > max_position_embeddings:
            current_input_ids = current_input_ids[:, current_seq_len - max_position_embeddings:]
            print(f"Warning: Sequence truncated from {current_seq_len} to {current_input_ids.shape[1]} for max_position_embeddings.")
            current_seq_len = current_input_ids.shape[1] # Update after truncation

        # --- Embeddings ---
        model.model.decoder.embed_tokens.to('cpu')
        # Use current_input_ids for embedding lookup
        x_embeddings = model.model.decoder.embed_tokens(current_input_ids.to('cpu')) # x_embeddings is on CPU
        model.model.decoder.embed_tokens.to('cpu') # Move layer back
        gc.collect()
        
        # 2. Add Positional Embeddings
        model.model.decoder.embed_positions.to('cpu')
        try:
            position_offset = model.model.decoder.embed_positions.offset
        except AttributeError:
            position_offset = 2 # Common default for OPT

        pos_ids = torch.arange(position_offset, position_offset + current_seq_len, device='cpu').unsqueeze(0) 
        
        # The main `x` variable throughout the layer processing will be `current_token_embeddings_plus_pos`
        current_token_embeddings_plus_pos = x_embeddings + model.model.decoder.embed_positions(pos_ids) 
        model.model.decoder.embed_positions.to('cpu') 
        del x_embeddings 
        del pos_ids 
        gc.collect()

        # --- Decoder Layers Loop ---
        # Rename 'x' to something that reflects its current state (embeddings in, embeddings out)
        x_current_layer_output = current_token_embeddings_plus_pos # Start of layer processing
        for j, layer in enumerate(model.model.decoder.layers):
            try:
                layer.to('cpu')
                x_current_layer_output = x_current_layer_output.to('cpu') 

                with torch.no_grad():
                    x_current_layer_output = layer(x_current_layer_output)[0]

                layer.to('cpu')
                x_current_layer_output = x_current_layer_output.to('cpu') 
                
                gc.collect()

            except Exception as e:
                print(f"Error in decoder layer {j}: {e}")
                break
        else: 
            # --- LM Head and Token Generation ---
            model.lm_head.to('cpu')
            # The input to lm_head is x_current_layer_output (the final embeddings from decoder)
            logits = model.lm_head(x_current_layer_output.to('cpu')) 
            model.lm_head.to('cpu') 
            
            logits = logits.to('cpu') # Ensure logits are on CPU
            gc.collect()

            # Get the predicted token ID (on CPU)
            # Take the last token's prediction from batch 0
            predicted_token_id = logits.argmax(-1)[:, -1].item() 

            decoded_token = tok.decode(predicted_token_id)
            print(decoded_token, end=" ")
            
            # --- Prepare input_ids_for_next_step for the next iteration ---
            # Append the new token ID to the input_ids_for_next_step tensor (on CPU)
            new_token_tensor = torch.tensor([[predicted_token_id]], dtype=torch.long, device='cpu')
            
            # This is the correct concatenation for input_ids
            input_ids_for_next_step = torch.cat((input_ids_for_next_step, new_token_tensor), dim=1)
            
            # Update the 'question' string for printing and future reference
            question = question + " " + decoded_token
            
        if 'predicted_token_id' not in locals(): # If inner loop broke, stop outer loop
            break

In [ ]:
def model_inference(question,tokens=250):
    for i in range(tokens):
        prompt=tok(question,return_tensors='pt').to('cuda')
        input_ids=prompt['input_ids']
        del prompt
        torch.cuda.empty_cache()
        model.model.decoder.embed_tokens.to('cuda')
        x = model.model.decoder.embed_tokens(input_ids)
        del input_ids
        torch.cuda.empty_cache()
        model.model.decoder.embed_tokens.to('cpu')
        # del model.model.decoder.embed_tokens
        torch.cuda.empty_cache()
        model.model.decoder.embed_positions.to('cuda')
        x = x + model.model.decoder.embed_positions(torch.arange(x.shape[1]).unsqueeze(0).to('cuda'))
        model.model.decoder.embed_positions.to('cpu')
        # del model.model.decoder.embed_positions
        torch.cuda.empty_cache()
        for i in model.model.decoder.layers:
            try:
                i.to('cuda')
                x=x.to('cuda')
                with torch.no_grad():
                    x=i(x)[0]
                i.to('cpu')
                torch.cuda.empty_cache()
                x=x.to('cpu')
                torch.cuda.empty_cache()
                # time.sleep(3)
            except:
                print(x)
                break
        model.lm_head.to('cuda')
        x=model.lm_head(x.to('cuda'))
        model.lm_head.to('cpu')
        torch.cuda.empty_cache()
        x1=x.argmax(-1)
        print(tok.decode(x1[0]),end=" ")
        question=question+" "+tok.decode(x1[0])
                

In [ ]:
model_inference("How are you")

.''. your retarded're .''. you retarded're .dylib mom retard.''. 

NameError: name 'e' is not defined

In [ ]:
print()
cnt=0
for i in model.model.decoder.layers:
    cnt+=1
    print(cnt)
    try:
        i.to('cuda')
        x.to('cuda')
        with torch.no_grad():
            x=i(x)[0]
        i.to('cpu')
        torch.cuda.empty_cache()
        x.to('cpu')
        torch.cuda.empty_cache()
        # time.sleep(3)
    except:
        print(cnt)
        print(x)
        break


1
2
3
4
5
6
7
8
9
10
11
12


In [ ]:
model.lm_head.to('cuda')
x=model.lm_head(x.to('cuda'))
model.lm_head.to('cpu')
torch.cuda.empty_cache()

In [ ]:
x1=x.argmax(-1)

In [ ]:
tok.decode(x1[0])

' hi'

# 3 Workers

In [1]:
import torch
from transformers import AutoModelForCausalLM,AutoTokenizer,BitsAndBytesConfig
from torch import nn
import time
import gc

# Add at the beginning of your script:
torch.backends.cudnn.benchmark = True
torch.backends.cuda.matmul.allow_tf32 = True

bnb_config=BitsAndBytesConfig(
            load_in_8bit=True,
            # bnb_4bit_compute_dtype=torch.bfloat16,
            # bnb_4bit_use_double_quant=True,
            # bnb_8bit_quant_type='nf4',
        )

model=AutoModelForCausalLM.from_pretrained("meta-llama/Llama-3.2-1B",
                                           torch_dtype=torch.bfloat16,
                                           use_cache=True,
                                        #    quantization_config=bnb_config
                                           )

# print(model)
tok=AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-1B")

def get_layers(model,drop=2):
    device_memory=(torch.cuda.get_device_properties('cuda').total_memory)/(1024**3)
    l=model.model.layers[0]
    num_params = sum(p.numel() for p in l.parameters())
    param_size_bytes = num_params * 2
    param_size_GB = param_size_bytes / (1024 ** 3)
    return int(device_memory/param_size_GB)-drop

class Module(nn.Module):
    def __init__(self, layers, i, j):
        super().__init__()
        self.model = nn.ModuleList(layers[i:j])
       
    @torch.compile
    def forward(self, x, cos, sin,attention_mask):
        for layer in self.model:
            x = layer(x, position_embeddings=(cos, sin))[0]
        return x

layers=model.model.layers

def compute(model_instance, x, layers_per_chunk, cos, sin, k, max_new_tokens, attention_mask):
    total_layers = len(model_instance.model.layers)
    stream_compute = torch.cuda.Stream()
    stream_transfer = torch.cuda.Stream()
    stream_offload = torch.cuda.Stream()
    
    if layers_per_chunk >= total_layers:
        i = 0
        l = total_layers
        m = Module(model_instance.model.layers, i, i + l).to('cuda')
        x = m(x.to('cuda'), cos.to('cuda'), sin.to('cuda'), attention_mask.to('cuda'))
        if k == max_new_tokens - 1:
            m = m.to('cpu')
            torch.cuda.empty_cache()
        return x

    i = 0
    rem = total_layers
    m_current = None
    m_next = None
    m_previous = None

    # Load first chunk
    if rem > 0:
        l_first = min(layers_per_chunk, rem)
        with torch.cuda.stream(stream_transfer):
            m_next = Module(model_instance.model.layers, i, i + l_first).to('cuda', non_blocking=True)
        i += l_first
        rem -= l_first

    while True:
        # Wait for transfer to complete before starting computation
        stream_compute.wait_stream(stream_transfer)
        
        # Move pointers
        m_previous = m_current
        m_current = m_next
        m_next = None

        # Compute on current chunk
        with torch.cuda.stream(stream_compute):
            with torch.no_grad():
                x = m_current(x.to('cuda', non_blocking=True),
                              cos.to('cuda', non_blocking=True),
                              sin.to('cuda', non_blocking=True),
                              attention_mask.to('cuda', non_blocking=True))

        # Offload previous chunk to CPU (if exists)
        if m_previous is not None:
            stream_offload.wait_stream(stream_compute)  # Wait for computation to finish before offloading
            with torch.cuda.stream(stream_offload):
                m_previous = m_previous.to('cpu', non_blocking=True)
                del m_previous
            torch.cuda.empty_cache()

        # If this is the last chunk, break
        if rem == 0:
            stream_compute.synchronize()
            # Offload the final chunk
            with torch.cuda.stream(stream_offload):
                m_current = m_current.to('cpu', non_blocking=True)
            stream_offload.synchronize()
            torch.cuda.empty_cache()
            break

        # Load next chunk
        with torch.cuda.stream(stream_transfer):
            l_next = min(layers_per_chunk, rem)
            m_next = Module(model_instance.model.layers, i, i + l_next).to('cuda', non_blocking=True)

        i += l_next
        rem -= l_next

    return x

def model_inference(question,tokens=250):
    prompt=tok(question,return_tensors='pt').to('cpu')
    input_ids=prompt['input_ids'].to('cuda',non_blocking=True)
    attention_mask=prompt['attention_mask']
    model.model.embed_tokens.to('cuda')
    model.model.norm.to('cuda')
    model.lm_head.to('cuda')
    # x = model.model.embed_tokens(input_ids)
    rotary_emb = model.model.rotary_emb
   
    del prompt
    for k in range(tokens):
        torch.cuda.empty_cache()
        # model.model.embed_tokens.to('cuda')
        # if i!=0:
        x = model.model.embed_tokens(input_ids)
        seq_len = x.shape[1]
        batch_size = x.shape[0]
        position_ids = torch.arange(seq_len, dtype=torch.long, device=x.device).unsqueeze(0).expand(batch_size, -1)
       
        cos, sin = rotary_emb(x=x,position_ids=position_ids)
        # input_ids.to('cpu')
        # torch.cuda.empty_cache()
        # model.model.embed_tokens.to('cpu')
        # torch.cuda.empty_cache()
       
        cnt=0
        with torch.no_grad():
            l1=get_layers(model,7)
            x=compute(model,x,2,cos,sin,k,tokens,attention_mask)
            # model.model.norm.to('cuda')
            x=model.model.norm(x)
            # model.model.norm.to('cpu')
            # torch.cuda.empty_cache()
            # model.lm_head.to('cuda')
            x=model.lm_head(x)
            # model.lm_head.to('cpu')
            # torch.cuda.empty_cache()
       
        x1=x[:,-1,:]
        x1=torch.argmax(x1,dim=-1,keepdim=True)
        print(tok.decode(x1[0]),end="")
        input_ids=torch.cat([input_ids,x1],dim=-1)
        # x1=x.argmax(-1)
        # print(tok.decode(x1[0]),end="\n")
        # question=question+" "+tok.decode(x1[0])
    



/home/pa/miniconda3/envs/gen_ai/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
start=time.time()
model_inference("HI",10)
model.to('cpu')
print()
print("Total time",time.time()-start)

W0819 09:52:38.438194 1123664 site-packages/torch/_inductor/utils.py:1250] [0/0] Not enough SMs to use max_autotune_gemm mode
/home/pa/miniconda3/envs/gen_ai/lib/python3.10/site-packages/torch/_inductor/lowering.py:7007: UserWarning: 
Online softmax is disabled on the fly since Inductor decides to
split the reduction. Cut an issue to PyTorch if this is an
important use case and you want to speed it up with online
softmax.

  warnings.warn(
/home/pa/miniconda3/envs/gen_ai/lib/python3.10/site-packages/torch/_inductor/compile_fx.py:2177: UserWarning: NVIDIA T1000 8GB does not support bfloat16 compilation natively, skipping
  warnings.warn(
/home/pa/miniconda3/envs/gen_ai/lib/python3.10/site-packages/torch/_inductor/lowering.py:7007: UserWarning: 
Online softmax is disabled on the fly since Inductor decides to
split the reduction. Cut an issue to PyTorch if this is an
important use case and you want to speed it up with online
softmax.

  warnings.warn(
/home/pa/miniconda3/envs/gen_ai/lib/p

, I am a 19 year old girl who
Total time 10.8710618019104


In [3]:
start=time.time()
model_inference("HI",10)
print()
print("Total time",time.time()-start)

, I am a 19 year old girl who
Total time 4.613088607788086


In [2]:
start=time.time()
model_inference("HI",10)
print()
print("Total time",time.time()-start)

W0809 09:47:38.365181 121776 site-packages/torch/_inductor/utils.py:1250] [0/0] Not enough SMs to use max_autotune_gemm mode
/home/pa/miniconda3/envs/gen_ai/lib/python3.10/site-packages/torch/_inductor/lowering.py:7007: UserWarning: 
Online softmax is disabled on the fly since Inductor decides to
split the reduction. Cut an issue to PyTorch if this is an
important use case and you want to speed it up with online
softmax.

  warnings.warn(
/home/pa/miniconda3/envs/gen_ai/lib/python3.10/site-packages/torch/_inductor/compile_fx.py:2177: UserWarning: NVIDIA T1000 8GB does not support bfloat16 compilation natively, skipping
  warnings.warn(
/home/pa/miniconda3/envs/gen_ai/lib/python3.10/site-packages/torch/_inductor/lowering.py:7007: UserWarning: 
Online softmax is disabled on the fly since Inductor decides to
split the reduction. Cut an issue to PyTorch if this is an
important use case and you want to speed it up with online
softmax.

  warnings.warn(
/home/pa/miniconda3/envs/gen_ai/lib/py

, I am a 19 year old girl who
Total time 18.63120675086975


In [ ]:
start=time.time()
model_inference("HI",10)
print()
print("Total time",time.time()-start)

, I am a 19 year old girl who
Total time 4.665896415710449


# 4 Workers 


In [5]:
from transformers.models.llama.modeling_llama import DynamicCache

In [4]:
pip install transformers==4.53.2

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 23.5 MB/s eta 0:00:00 0:00:01
  Attempting uninstall: transformers
    Found existing installation: transformers 4.55.0
    Uninstalling transformers-4.55.0:
      Successfully uninstalled transformers-4.55.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
vllm 0.10.1.1 requires transformers>=4.55.0, but you have transformers 4.53.2 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [1]:
import torch
import os
from transformers import AutoModelForCausalLM,AutoTokenizer,BitsAndBytesConfig
from torch import nn
import time
import gc
from transformers.models.llama.modeling_llama import DynamicCache

os.environ["FLASH_ATTENTION_FORCE_DISABLED"] = "1"
torch.set_num_threads(6)
torch.set_num_interop_threads(3)

torch.backends.cudnn.benchmark = True
torch.backends.cuda.matmul.allow_tf32 = True

bnb_config=BitsAndBytesConfig(
            load_in_4bit=True,
            # bnb_4bit_compute_dtype=torch.bfloat16,
            # bnb_4bit_use_double_quant=True,
            # bnb_8bit_quant_type='nf4',
        )

model=AutoModelForCausalLM.from_pretrained("meta-llama/Llama-3.2-1B",
                                           torch_dtype=torch.bfloat16,
                                           trust_remote_code=True,
                                           use_cache=True,
                                        #    quantization_config=bnb_config
                                           )

# print(model)
model.eval()
tok=AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-1B")
tokenizer=tok

/home/pa/miniconda3/envs/gen_ai/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#gpu efficient

def get_layers(model,drop=2):
    device_memory=(torch.cuda.get_device_properties('cuda').total_memory)/(1024**3)
    l=model.model.layers[0]
    num_params = sum(p.numel() for p in l.parameters())
    param_size_bytes = num_params * 2
    param_size_GB = param_size_bytes / (1024 ** 3)
    return (int(device_memory/param_size_GB)-drop)//2

class Module(nn.Module):
    def __init__(self, layers, i, j):
        super().__init__()
        self.model = nn.ModuleList(layers[i:j])
    
    # @torch.compile
    def forward(self, x, cos, sin,dynamic_cache,attention_mask):
        for layer in self.model:
            x = layer(x, position_embeddings=(cos, sin),past_key_value=dynamic_cache,use_cache=True,)[0]
        return x
tokenizer=tok
layers=model.model.layers

def compute(model_instance, x, layers_per_chunk, cos, sin, k, dynamic_cache,max_new_tokens, attention_mask,modules):
    total_layers = len(model_instance.model.layers)
    stream_compute = torch.cuda.Stream()
    stream_transfer = torch.cuda.Stream()
    stream_transfer2 = torch.cuda.Stream()
    # stream_transfer3 = torch.cuda.Stream()  # New 3rd stream
    stream_offload = torch.cuda.Stream()
    
    if layers_per_chunk >= total_layers:
        i = 0
        l = total_layers
        m = Module(model_instance.model.layers, i, i + l).to('cuda', non_blocking=True)
        with torch.no_grad():
            x = m(x, cos, sin,dynamic_cache,attention_mask)
        if k == max_new_tokens - 1:
            m = m.to('cpu', non_blocking=True)
        return x

    # i = 0
    
    m_current = None
    m_current2=None
    m_next = None
    m_next2 = None
    m_previous = None
    m_previous2=None
    rem = len(modules)
    i=0
    if rem > 0:
        with torch.cuda.stream(stream_transfer):
            full_mod = modules[i]  # This is already your Module wrapper
            total_layers_in_mod = len(full_mod.model)
            half = total_layers_in_mod // 2

            # m_next = Module(full_mod.model, 0, half).to('cuda', non_blocking=True)
        # with torch.cuda.stream(stream_transfer2):
            m_next2 = Module(full_mod.model, half, total_layers_in_mod).to('cuda', non_blocking=True)
        i += 1
        rem -= 1
    stream_compute.wait_stream(stream_transfer)
    while True:
        stream_compute.wait_stream(stream_transfer)

        m_previous = m_previous2
        m_previous2=m_current
        m_current = m_next
        # m_current2 = m_next2
        m_next=m_next2
        m_next2 = None
        
        
        with torch.cuda.stream(stream_compute):
            with torch.no_grad():
                if(m_current!=None):
                    x = m_current(
                        x,
                        cos,
                        sin,
                        dynamic_cache,
                        attention_mask
                    )
        if m_previous is not None:
            with torch.cuda.stream(stream_offload):
                m_previous.to('cpu',non_blocking=True)
        # if m_previous2 is not None:
        #     with torch.cuda.stream(stream_offload):
        #         m_previous2.to('cpu',non_blocking=True)
        torch.cuda.empty_cache()
        if rem == 0 and m_next is None and m_next2 is None :
            if k==max_new_tokens-1:
                stream_compute.synchronize()
                with torch.cuda.stream(stream_offload):
                    m_current = m_current.to('cpu', non_blocking=True)
                stream_offload.synchronize()
                torch.cuda.empty_cache()
                break
            break
        if rem > 0:
            with torch.cuda.stream(stream_transfer):
                full_mod = modules[i]  # This is already your Module wrapper
                total_layers_in_mod = len(full_mod.model)
                half = total_layers_in_mod // 2

                # m_next = Module(full_mod.model, 0, half).to('cuda', non_blocking=True)
            # with torch.cuda.stream(stream_transfer2):
                m_next2 = Module(full_mod.model, half, total_layers_in_mod).to('cuda', non_blocking=True)
            i += 1
            rem -= 1
    return x


# Initialize graph variables as function attributes



def model_inference(question,tokens=250):
    prompt=tok(question,return_tensors='pt').to('cpu',non_blocking=True)
    input_ids=prompt['input_ids'].to('cuda',non_blocking=True)
    attention_mask=prompt['attention_mask']
    model.model.embed_tokens.to('cuda',non_blocking=True)
    model.model.norm.to('cuda',non_blocking=True)
    model.lm_head.to('cuda',non_blocking=True)
    rotary_emb = model.model.rotary_emb.to('cuda',non_blocking=True)
    dynamic_cache = DynamicCache()
    current_input_ids = input_ids
    modules=[]
    rem1=len(model.model.layers)
    i=0
    while(rem1>=1):
        l=min(1,rem1)
        modules.append(Module(model.model.layers,i,i+l).to('cpu',non_blocking=True))
        i+=l
        rem1-=l
    for k in range(tokens):
        if k==0:
            seq_len = current_input_ids.shape[1]
            batch_size = current_input_ids.shape[0]
            current_position_id=torch.arange(seq_len, dtype=torch.long, device=current_input_ids.device).unsqueeze(0).expand(batch_size, -1)
        else:
            current_position_id = torch.tensor([[dynamic_cache.get_seq_length()]], device='cuda')
        x = model.model.embed_tokens(current_input_ids)
        cos, sin = rotary_emb(x=x, position_ids=current_position_id)
        with torch.no_grad():
            l1=get_layers(model,7)
            x=compute(model,x,2,cos,sin,k,dynamic_cache,tokens,attention_mask,modules)
            x=model.model.norm(x)
            logits=model.lm_head(x)
        next_token_id = torch.argmax(logits[:, -1, :], dim=-1, keepdim=True)
        print(tokenizer.decode(next_token_id[0]), end="")
        current_input_ids = next_token_id
    model.model.embed_tokens.to('cpu',non_blocking=True)
    torch.cuda.empty_cache()
    model.model.norm.to('cpu',non_blocking=True)
    torch.cuda.empty_cache()
    model.lm_head.to('cpu',non_blocking=True)
    torch.cuda.empty_cache()
    rotary_emb.to('cpu',non_blocking=True)
    torch.cuda.empty_cache()

In [ ]:
start=time.time()
model_inference("tell", 100)
print()
print(time.time()-start) #2640 2474 2172

 me about the best way to get to the beach from the airport
I am planning a trip to the beach in the next few weeks and I am wondering what is the best way to get to the beach from the airport.

In [5]:
pip install transformers==4.55.0

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 25.2 MB/s eta 0:00:00 0:00:01
  Attempting uninstall: transformers
    Found existing installation: transformers 4.55.4
    Uninstalling transformers-4.55.4:
      Successfully uninstalled transformers-4.55.4
Note: you may need to restart the kernel to use updated packages.


In [6]:
start=time.time()
model_inference("hello",100)
print()
print(time.time()-start) #2640 2474 2172

, i am new to this forum and i am trying to get a better understanding of the different types of data that can be used in a regression model. i am trying to understand the difference between the following types of data:
1) nominal data (e.g. gender, race, etc.)
2) ordinal data (e.g. age, income, etc.)
3) interval data (e.g. weight, height, etc.)
4) ratio data (e.g. income, weight, etc
24.25179409980774


In [5]:
model_chunker = ModelChunker(model, layers_per_chunk=4)  # 4 layers per chunk

# Now you can run inference multiple times without rebuilding chunks
model_inference_optimized(model_chunker, "Hi", tokens=100)

Building 16 layers into chunks of 4 layers each...
Building chunk 0: layers 0 to 3
Building chunk 1: layers 4 to 7
Building chunk 2: layers 8 to 11
Building chunk 3: layers 12 to 15
Successfully built 4 chunks
, I'm a new user and I'm trying to get my first project to work. I'm using the latest version of the Arduino IDE and the latest version of the Arduino IDE. I'm using the Arduino IDE to compile the code and upload it to the Arduino board. I'm using the Arduino IDE to compile the code and upload it to the Arduino board. I'm using the Arduino IDE to compile the code and upload it to the Arduino board. I'm using the Arduino IDE to compile the

In [1]:
import torch
from transformers import AutoModelForCausalLM,AutoTokenizer,BitsAndBytesConfig
from torch import nn
import time
import gc
from transformers.models.llama.modeling_llama import DynamicCache
# Add at the beginning of your script:
torch.backends.cudnn.benchmark = True
torch.backends.cuda.matmul.allow_tf32 = True

bnb_config=BitsAndBytesConfig(
            load_in_8bit=True,
            # bnb_4bit_compute_dtype=torch.bfloat16,
            # bnb_4bit_use_double_quant=True,
            # bnb_8bit_quant_type='nf4',
        )

model=AutoModelForCausalLM.from_pretrained("meta-llama/Llama-3.2-1B",
                                           torch_dtype=torch.bfloat16,
                                          #  use_cache=True,
                                          #  attn_implementation="eager",
                                          #  force_install=True
                                          #  rope_scaling={'type':"llama3",'factor':32.0}
                                          #  offload_folder='/offload_nvm',
                                          #  offload_state_dict=True,
                                          #  max_memory={"cpu":'0GB'},
                                           quantization_config=bnb_config
                                           )

# print(model)
tok=AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-1B")

for name, module in model.named_modules():
    for p in module.parameters():
        if p.device.type == 'cpu':
            p.data = p.data.pin_memory()
            
# Test if your parameters are actually pinned
# def check_if_pinned(model):
#     for name, param in model.named_parameters():
#         if param.device.type == 'cpu':
#             print(f"{name}: pinned = {param.is_pinned()}")

# check_if_pinned(model)  # Should show True for all parameters

/home/pa/miniconda3/envs/gen_ai/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm

A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.6 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "/home/pa/miniconda3/envs/gen_ai/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/home/pa/miniconda3/envs/gen_ai/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)


AttributeError: _ARRAY_API not found


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.6 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "/home/pa/miniconda3/envs/gen_ai/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/home/pa/miniconda3/envs/gen_ai/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/home/pa/miniconda3/envs/gen_ai/lib/python3.10/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/home/pa/miniconda3/envs/gen_ai/lib/python3.10/site-packages/traitlets/config/application.py", line 1075, in launc

AttributeError: _ARRAY_API not found

In [2]:
pip show transformers

Name: transformers
Version: 4.55.4
Summary: State-of-the-art Machine Learning for JAX, PyTorch and TensorFlow
Home-page: https://github.com/huggingface/transformers
Author: The Hugging Face team (past and future) with the help of all our contributors (https://github.com/huggingface/transformers/graphs/contributors)
Author-email: transformers@huggingface.co
License: Apache 2.0 License
Location: /home/pa/miniconda3/envs/gen_ai/lib/python3.10/site-packages
Requires: filelock, huggingface-hub, numpy, packaging, pyyaml, regex, requests, safetensors, tokenizers, tqdm
Required-by: compressed-tensors, sentence-transformers, vllm, xgrammar
Note: you may need to restart the kernel to use updated packages.


In [5]:
pip cache purge


Files removed: 2652 (18525.2 MB)
Note: you may need to restart the kernel to use updated packages.


In [2]:
class Module(nn.Module):
    def __init__(self, layers, i, j):
        super().__init__()
        self.model = nn.ModuleList(layers[i:j])
        # print(self.model.device)
       
    @torch.compile
    def forward(self, x, cos, sin,attention_mask):
        for layer in self.model:
            # print(layer.device)
            x = layer(x, position_embeddings=(cos, sin))[0]
        return x



def compute(model_instance, x, layers_per_chunk, cos, sin, k, max_new_tokens, attention_mask):
    total_layers = len(model_instance.model.layers)
    stream_compute = torch.cuda.Stream()
    stream_transfer = torch.cuda.Stream()
    stream_transfer2 = torch.cuda.Stream()
    stream_offload = torch.cuda.Stream()
    
    if layers_per_chunk >= total_layers:
        i = 0
        l = total_layers
        m = Module(model_instance.model.layers, i, i + l).to('cuda',non_blocking=True)
        x = m(x.to('cuda'), cos.to('cuda'), sin.to('cuda'), attention_mask.to('cuda'))
        if k == max_new_tokens - 1:
            m = m.to('cpu',non_blocking=True)
            torch.cuda.empty_cache()
        return x

    i = 0
    rem = total_layers
    m_current = None
    m_next = None
    m_next2 = None
    m_previous = None

    # Load first two chunks
    if rem > 0:
        l_first = min(layers_per_chunk, rem)
        with torch.cuda.stream(stream_transfer):
            m_next = Module(model_instance.model.layers, i, i + l_first).to('cuda', non_blocking=True)
        i += l_first
        rem -= l_first
        
        if rem > 0:
            l_second = min(layers_per_chunk, rem)
            with torch.cuda.stream(stream_transfer2):
                m_next2 = Module(model_instance.model.layers, i, i + l_second).to('cuda', non_blocking=True)
            i += l_second
            rem -= l_second
    while True:
    # Wait for previous transfer to complete before compute
        stream_compute.wait_stream(stream_transfer)

        # Move pointers forward
        m_previous = m_current
        m_current = m_next
        m_next = m_next2
        m_next2 = None

        # Compute on current chunk
        with torch.cuda.stream(stream_compute):
            with torch.no_grad():
                x = m_current(
                    x.to('cuda', non_blocking=True),
                    cos.to('cuda', non_blocking=True),
                    sin.to('cuda', non_blocking=True),
                    attention_mask.to('cuda', non_blocking=True)
                )

        # Offload the previous chunk after compute is done
        if m_previous is not None:
            stream_offload.wait_stream(stream_compute)
            with torch.cuda.stream(stream_offload):
                m_previous = m_previous.to('cpu', non_blocking=True)
                del m_previous
            torch.cuda.empty_cache()

        # If no more chunks left to load after this, finish
        if rem == 0 and m_next is None and m_next2 is None:
            # Wait for compute to complete
            stream_compute.synchronize()

            # Offload the current module
            with torch.cuda.stream(stream_offload):
                m_current = m_current.to('cpu', non_blocking=True)
                del m_current
            stream_offload.synchronize()
            torch.cuda.empty_cache()
            break

        # Start preloading next-next chunk if any
        if rem > 0:
            l_next2 = min(layers_per_chunk, rem)
            stream_transfer2.wait_stream(stream_transfer)
            with torch.cuda.stream(stream_transfer2):
                m_next2 = Module(model_instance.model.layers, i, i + l_next2).to('cuda', non_blocking=True)
            i += l_next2
            rem -= l_next2
    return x

def model_inference(question,tokens=250):
    prompt=tok(question,return_tensors='pt').to('cpu',non_blocking=True)
    input_ids=prompt['input_ids'].to('cuda',non_blocking=True)
    attention_mask=prompt['attention_mask']
    model.model.embed_tokens.to('cuda',non_blocking=True)
    model.model.norm.to('cuda',non_blocking=True)
    model.lm_head.to('cuda',non_blocking=True)
    x = model.model.embed_tokens(input_ids)
    rotary_emb = model.model.rotary_emb
   
    del prompt
    for k in range(tokens):
        torch.cuda.empty_cache()
        # model.model.embed_tokens.to('cuda',non_blocking=True)
        # if i!=0:
        x = model.model.embed_tokens(input_ids)
        seq_len = x.shape[1]
        batch_size = x.shape[0]
        position_ids = torch.arange(seq_len, dtype=torch.long, device=x.device).unsqueeze(0).expand(batch_size, -1)
       
        cos, sin = rotary_emb(x=x,position_ids=position_ids)
        # input_ids.to('cpu')
        # torch.cuda.empty_cache()
        # model.model.embed_tokens.to('cpu')
        # torch.cuda.empty_cache()
       
        cnt=0
        with torch.no_grad():
            # l1=get_layers(model,7)
            x=compute(model,x,2,cos,sin,k,tokens,attention_mask)
            # model.model.norm.to('cuda',non_blocking=True)
            x=model.model.norm(x)
            # model.model.norm.to('cpu',non_blocking=True)
            torch.cuda.empty_cache()
            # model.lm_head.to('cuda',non_blocking=True)
            x=model.lm_head(x)
            # model.lm_head.to('cpu',non_blocking=True)
            torch.cuda.empty_cache()
        
        x1=x[:,-1,:]
        x1=torch.argmax(x1,dim=-1,keepdim=True)
        print(tok.decode(x1[0]),end="")
        input_ids=torch.cat([input_ids,x1],dim=-1)
        # x1=x.argmax(-1)
        # print(tok.decode(x1[0]),end="\n")
        # question=question+" "+tok.decode(x1[0])
    input_ids.to('cpu')
    model.model.norm.to('cpu',non_blocking=True)
    torch.cuda.empty_cache()
    model.lm_head.to('cpu',non_blocking=True)
    torch.cuda.empty_cache()

In [3]:
start=time.time()
model_inference("Hello",100)
print()
print("Total time",time.time()-start)

W0813 09:31:51.247132 5599 site-packages/torch/_inductor/utils.py:1250] [0/0] Not enough SMs to use max_autotune_gemm mode
/home/pa/miniconda3/envs/gen_ai/lib/python3.10/site-packages/torch/_inductor/lowering.py:7007: UserWarning: 
Online softmax is disabled on the fly since Inductor decides to
split the reduction. Cut an issue to PyTorch if this is an
important use case and you want to speed it up with online
softmax.

  warnings.warn(
/home/pa/miniconda3/envs/gen_ai/lib/python3.10/site-packages/torch/_inductor/compile_fx.py:2177: UserWarning: NVIDIA T1000 8GB does not support bfloat16 compilation natively, skipping
  warnings.warn(
/home/pa/miniconda3/envs/gen_ai/lib/python3.10/site-packages/torch/_inductor/lowering.py:7007: UserWarning: 
Online softmax is disabled on the fly since Inductor decides to
split the reduction. Cut an issue to PyTorch if this is an
important use case and you want to speed it up with online
softmax.

  warnings.warn(
/home/pa/miniconda3/envs/gen_ai/lib/pyth

, I am interested in [Project 1]Design and analysis of a 2D Poisson's equation for the given data. I have a problem with the Poisson's equation. I have a problem with the Poisson's equation. I have a problem with the Poisson's equation. I have a problem with the Poisson's equation. I have a problem with the Poisson's equation. I have a problem with the Poisson's equation. I have a problem with the Po
Total time 42.96747446060181


In [13]:
def get_layers(model, drop=2):
    device_memory = (torch.cuda.get_device_properties('cuda').total_memory) / (1024 ** 3)
    l = model.model.layers[0]
    num_params = sum(p.numel() for p in l.parameters())
    param_size_bytes = num_params * 2
    param_size_GB = param_size_bytes / (1024 ** 3)
    return (int(device_memory / param_size_GB) - drop) // 2

class Module(nn.Module):
    def __init__(self, layers, i, j):
        super().__init__()
        self.model = nn.ModuleList(layers[i:j])

    # @torch.compile
    def forward(self, x, cos, sin, attention_mask):
        for layer in self.model:
            x = layer(x, position_embeddings=(cos, sin))[0]
        return x

layers = model.model.layers

def compute(model_instance, x, layers_per_chunk, cos, sin, k, max_new_tokens, attention_mask):
    total_layers = len(model_instance.model.layers)
    stream_compute = torch.cuda.Stream()
    stream_transfer = torch.cuda.Stream()
    stream_transfer2 = torch.cuda.Stream()
    stream_offload = torch.cuda.Stream()

    if layers_per_chunk >= total_layers:
        i = 0
        l = total_layers
        m = Module(model_instance.model.layers, i, i + l).to('cuda', non_blocking=True)
        # time.sleep(5.01)  # delay 1
        x = m(x.to('cuda'), cos.to('cuda'), sin.to('cuda'), attention_mask.to('cuda'))
        if k == max_new_tokens - 1:
            m = m.to('cpu', non_blocking=True)
            torch.cuda.empty_cache()
        return x

    i = 0
    rem = total_layers
    m_current = None
    m_next = None
    m_next2 = None
    m_previous = None

    if rem > 0:
        l_first = min(layers_per_chunk, rem)
        with torch.cuda.stream(stream_transfer):
            m_next = Module(model_instance.model.layers, i, i + l_first).to('cuda', non_blocking=True)
        # time.sleep(5.01)  # delay 2
        i += l_first
        rem -= l_first

        if rem > 0:
            l_second = min(layers_per_chunk, rem)
            with torch.cuda.stream(stream_transfer2):
                m_next2 = Module(model_instance.model.layers, i, i + l_second).to('cuda', non_blocking=True)
            # time.sleep(5.01)  # delay 3
            i += l_second
            rem -= l_second

    while True:
        stream_compute.wait_stream(stream_transfer)
        # time.sleep(5.01)  # delay 4

        m_previous = m_current
        m_current = m_next
        m_next = m_next2
        m_next2 = None

        # time.sleep(5.01)  # delay 5

        with torch.cuda.stream(stream_compute):
            # time.sleep(5.01)  # delay 6
            with torch.no_grad():
                x = m_current(x.to('cuda', non_blocking=True),
                              cos.to('cuda', non_blocking=True),
                              sin.to('cuda', non_blocking=True),
                              attention_mask.to('cuda', non_blocking=True))

        # time.sleep(5.01)  # delay 7

        if m_previous is not None:
            stream_offload.wait_stream(stream_compute)
            # time.sleep(5.01)  # delay 8
            with torch.cuda.stream(stream_offload):
                m_previous = m_previous.to('cpu', non_blocking=True)
                del m_previous
            # time.sleep(5.01)  # delay 9
            torch.cuda.empty_cache()

        if rem == 0:
            stream_compute.synchronize()
            # time.sleep(8)  # delay 10
            with torch.cuda.stream(stream_offload):
                m_current = m_current.to('cpu', non_blocking=True)
            stream_offload.synchronize()
            # time.sleep(8)  # delay 11
            torch.cuda.empty_cache()
            break

        if rem > 0:
            # print("runing test")
            time.sleep(0.01)  # delay 12
            l_next2 = min(layers_per_chunk, rem)
            with torch.cuda.stream(stream_transfer2):
                m_next2 = Module(model_instance.model.layers, i, i + l_next2).to('cuda', non_blocking=True)
            time.sleep(0.01)  # delay 13
            i += l_next2
            rem -= l_next2

    return x

def model_inference(question, tokens=250):
    prompt = tok(question, return_tensors='pt').to('cpu', non_blocking=True)
    input_ids = prompt['input_ids'].to('cuda', non_blocking=True)
    attention_mask = prompt['attention_mask']
    model.model.embed_tokens.to('cuda', non_blocking=True)
    model.model.norm.to('cuda', non_blocking=True)
    model.lm_head.to('cuda', non_blocking=True)
    # time.sleep(5.01)  # delay 14

    x = model.model.embed_tokens(input_ids)
    rotary_emb = model.model.rotary_emb

    del prompt
    for k in range(tokens):
        torch.cuda.empty_cache()
        # time.sleep(5.01)  # delay 15

        x = model.model.embed_tokens(input_ids)
        seq_len = x.shape[1]
        batch_size = x.shape[0]
        position_ids = torch.arange(seq_len, dtype=torch.long, device=x.device).unsqueeze(0).expand(batch_size, -1)

        cos, sin = rotary_emb(x=x, position_ids=position_ids)
        # time.sleep(5.01)  # delay 16

        with torch.no_grad():
            l1 = get_layers(model, 7)
            x = compute(model, x, 3, cos, sin, k, tokens, attention_mask)
            # time.sleep(5.01)  # delay 17

            x = model.model.norm(x)
            # time.sleep(5.01)  # delay 18

            x = model.lm_head(x)
            # time.sleep(5.01)/  # delay 19

        x1 = x[:, -1, :]
        x1 = torch.argmax(x1, dim=-1, keepdim=True)
        print(tok.decode(x1[0]), end="")
        input_ids = torch.cat([input_ids, x1], dim=-1)
        # time.sleep(5.01)  # delay 20

    input_ids.to('cpu')
    model.model.norm.to('cpu', non_blocking=True)
    torch.cuda.empty_cache()
    model.lm_head.to('cpu', non_blocking=True)
    torch.cuda.empty_cache()


In [ ]:
start=time.time()
model_inference("HI",100)
print()
print("Total time",time.time()-start)

 guys, please tell me this is a good deal
Total time 10.169853687286377


In [4]:
!pip install numpy==1.24.3 scipy==1.10.1
!pip install transformers==4.53.2 --force-reinstall

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 22.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.4/34.4 MB 16.4 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: numpy
    Found existing installation: numpy 1.25.2
    Uninstalling numpy-1.25.2:
      Successfully uninstalled numpy-1.25.2
  Attempting uninstall: scipy━━━━━━━━━━━━━━━━━━━ 0/2 [numpy]
    Found existing installation: scipy 1.11.1 0/2 [numpy]
    Uninstalling scipy-1.11.1:0m╺━━━━━━━━━━━━━━━━━━━ 1/2 [scipy]
      Successfully uninstalled scipy-1.11.1━━━━━━━━━━━━━━━━━━━ 1/2 [scipy]
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [scipy]32m1/2 [scipy]
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
faiss-cpu 1.11.0 requires numpy<3.0,>=1.25.0, but you have numpy 1.24.3 which is incompatible.
langchain-community 0.3.27 requires numpy>=1.26.2; python_version < "3.1

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Using cached transformers-4.53.2-py3-none-any.whl.metadata (40 kB)
  Using cached filelock-3.18.0-py3-none-any.whl.metadata (2.9 kB)
  Using cached huggingface_hub-0.34.4-py3-none-any.whl.metadata (14 kB)
  Using cached numpy-2.2.6-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (62 kB)
  Using cached packaging-25.0-py3-none-any.whl.metadata (3.3 kB)
  Using cached PyYAML-6.0.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (2.1 kB)
  Using cached regex-2025.7.34-cp310-cp310-manylinux2014_x86_64.manylinux_2_17_x86_64.manylinux_2_28_x86_64.whl.metadata (40 kB)
  Using cached requests-2.32.4-py3-none-any.whl.metadata (4.9 kB)
  Using cached tokenizers-0.21.4-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.7 kB)
  Using cached safetensors-0.6.2-cp38-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (4.1 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached typing_extensions-4.14.1-py3-none